# SD Scripts Kaggle
Created by [licyk](https://github.com/licyk)

Jupyter Notebook 仓库：[licyk/sd-webui-all-in-one](https://github.com/licyk/sd-webui-all-in-one)


## 简介
一个在 [Kaggle](https://www.kaggle.com) 部署 [sd-scripts](https://github.com/kohya-ss/sd-scripts) 的 Jupyter Notebook，可用于 Stable Diffusion 模型的训练。


## 不同运行单元的功能
该 Notebook 分为以下几个单元：

- [功能初始化](#功能初始化)
- [参数配置](#参数配置)
- [安装环境](#安装环境)
- [模型训练](#模型训练)
- [模型上传](#模型上传)

使用时请按顺序运行笔记单元。

通常情况下[功能初始化](#功能初始化)和[模型上传](#模型上传)单元的内容无需修改，其他单元包含不同功能的注释，可阅读注释获得帮助。

[参数配置](#参数配置)单元用于修改安装，训练，上传模型时的配置。

[安装](#安装)单元执行安装训练环境的命令和下载模型 / 训练集的命令，可根据需求进行修改。

[模型训练](#模型训练)执行训练模型的命令，需要根据自己的需求进行修改，该单元也提供一些训练参数的例子，可在例子的基础上进行修改。

如果需要快速取消注释，可以选中代码，按下`Ctrl + /`取消注释。


## 提示
1. 不同单元中包含注释, 可阅读注释获得帮助。
2. 训练代码的部分需要根据自己的需求进行更改。
3. 推荐使用 Kaggle 的 `Save Version` 的功能运行笔记，可让 Kaggle 笔记在无人值守下保持运行，直至所有单元运行完成。
4. 如果有 [HuggingFace](https://huggingface.co) 账号或者 [ModelScope](https://modelscope.cn) 账号，可通过填写 Token 和仓库名后实现自动上传训练好的模型，仓库需要手动创建。
5. 进入 Kaggle 笔记后，在 Kaggle 的右侧栏可以调整 kaggle 笔记的设置，也可以上传训练集等。注意，在 Kaggle 笔记的`Session options`->`ACCELERATOR`中，需要选择`GPU T4 x 2`，才能使用 GPU 进行模型训练。
6. 使用 Kaggle 进行模型训练时，训练集中最好没有 NSFW 内容，否则可能会导致 Kaggle 账号被封禁。
7. 不同单元的标题下方包含快捷跳转链接，可使用跳转链接翻阅 Notebook。
8. 该 Notebook 的使用方法可阅读：</br>[使用 HuggingFace / ModelScope 保存和下载文件 - licyk的小窝](https://licyk.netlify.app/2025/01/16/use-huggingface-or-modelscope-to-save-file/)</br>[使用 Kaggle 进行模型训练 - licyk的小窝](https://licyk.netlify.app/2025/01/16/use-kaggle-to-training-sd-model)


## 功能初始化
通常不需要修改该单元的内容  
1. [[下一个单元 →](#参数配置)]

In [ ]:
# SD Scripts Manager 功能初始化部分, 通常不需要修改
# 如果需要查看完整代码实现, 可阅读: https://github.com/licyk/sd-webui-all-in-one/blob/main/sd_scripts_ipynb_core.py
#################################################################################################################
# SD_SCRIPTS_IPYNB_CORE_URL, FORCE_DOWNLOAD_CORE 参数可根据需求修改, 通常保持默认即可
SD_SCRIPTS_IPYNB_CORE_URL = "https://github.com/licyk/sd-webui-all-in-one/raw/main/sd_scripts_ipynb_core.py" # SD Scripts Manager 核心下载地址
FORCE_DOWNLOAD_CORE = False # 设置为 True 时, 即使 SD Scripts Manager 已存在也会重新下载
#################################################################################################################
import os
import sys
import urllib.request
from pathlib import Path
try:
    print(f"[SD Scripts Manager] Jupyter Notebook 根目录: {JUPYTER_ROOT_PATH}") # type: ignore
except Exception as _:
    JUPYTER_ROOT_PATH = Path(os.getcwd())
    sys.path.append(str(JUPYTER_ROOT_PATH.resolve()))
    print(f"[SD Scripts Manager] Jupyter Notebook 根目录: {JUPYTER_ROOT_PATH}")
    SD_SCRIPTS_IPYNB_CORE_PATH = JUPYTER_ROOT_PATH / "sd_scripts_ipynb_core.py" # SD Scripts Manager 核心保存路径
try:
    if SD_SCRIPTS_IPYNB_CORE_PATH.exists() and not FORCE_DOWNLOAD_CORE:
        print("[SD Scripts Manager] SD Scripts Manager 核心模块已存在")
    else:
        urllib.request.urlretrieve(SD_SCRIPTS_IPYNB_CORE_URL, SD_SCRIPTS_IPYNB_CORE_PATH)
        print("[SD Scripts Manager] SD Scripts Manager 核心模块下载成功")
except Exception as e:
    raise Exception(f"SD Scripts Manager 核心模块下载错误: {e}")
from sd_scripts_ipynb_core import logger, VERSION, SDScriptsManager
logger.info("SD Scripts Manager 版本: %s", VERSION)
logger.info("核心模块初始化完成")

## 参数配置
设置必要的参数, 根据注释说明进行修改  
2. [[← 上一个单元](#功能初始化)|[下一个单元 →](#安装环境)]

In [ ]:
# 环境设置
WORKSPACE = "/kaggle" # 工作路径, 通常不需要修改
WORKFOLDER = "sd-scripts" # 工作路径中文件夹名称, 通常不需要修改
SD_SCRIPTS_REPO = "https://github.com/kohya-ss/sd-scripts" # sd-scripts 仓库地址
SD_SCRIPTS_REQUIREMENT = "requirements.txt" # sd-scripts 依赖文件名
TORCH_VER = "torch==2.5.1+cu124 torchvision==0.20.1+cu124 torchaudio==2.5.1+cu124" # PyTorch 版本
XFORMERS_VER = "xformers==0.0.28.post3" # xFormers 版本
USE_UV = True # 使用 uv 加速 Python 软件包安装, 修改为 True 为启用, False 为禁用
PIP_INDEX_MIRROR = "https://pypi.python.org/simple" # PyPI 主镜像源
PIP_EXTRA_INDEX_MIRROR = "https://download.pytorch.org/whl/cu124" # PyPI 扩展镜像源
PYTORCH_MIRROR = "https://download.pytorch.org/whl/cu124" # 用于下载 PyTorch 的镜像源
PIP_FIND_LINKS_MIRROR = "https://download.pytorch.org/whl/cu121/torch_stable.html" # PyPI 扩展镜像源
HUGGINGFACE_MIRROR = "https://hf-mirror.com" # HuggingFace 镜像源
GITHUB_MIRROR = [ # Github 镜像源
    "https://ghfast.top/https://github.com",
    "https://mirror.ghproxy.com/https://github.com",
    "https://ghproxy.net/https://github.com",
    "https://gh.api.99988866.xyz/https://github.com",
    "https://gh-proxy.com/https://github.com",
    "https://ghps.cc/https://github.com",
    "https://gh.idayer.com/https://github.com",
    "https://ghproxy.1888866.xyz/github.com",
    "https://slink.ltd/https://github.com",
    "https://github.boki.moe/github.com",
    "https://github.moeyy.xyz/https://github.com",
    "https://gh-proxy.net/https://github.com",
    "https://gh-proxy.ygxz.in/https://github.com",
    "https://wget.la/https://github.com",
    "https://kkgithub.com",
    "https://gitclone.com/github.com",
]
CHECK_AVALIABLE_GPU = False # 检查可用的 GPU, 当 GPU 不可用时强制终止安装进程
RETRY = 3 # 重试下载次数
DOWNLOAD_THREAD = 16 # 下载线程
ENABLE_TCMALLOC = True # 启用 TCMalloc 内存优化
ENABLE_CUDA_MALLOC = True # 启用 CUDA Malloc 显存优化

##############################################################################

# sd-scripts 版本设置
SD_SCRIPTS_BRANCH = "dev" # sd-scripts 分支, 可切换成 main / dev 或者其它分支, 留空则不进行切换
SD_SCRIPTS_COMMIT = "" # 切换 sd-scripts 的版本到某个 Git 提交记录上, 留空则不进行切换

##############################################################################

# 模型上传设置, 使用 HuggingFace / ModelScope 上传训练好的模型
# HuggingFace: https://huggingface.co
# ModelScope: https://modelscope.cn
USE_HF_TO_SAVE_MODEL = False # 使用 HuggingFace 保存训练好的模型, 修改为 True 为启用, False 为禁用 (True / False)
USE_MS_TO_SAVE_MODEL = False # 使用 ModelScope 保存训练好的模型, 修改为 True 为启用, False 为禁用 (True / False)

# Token 配置, 用于上传 / 下载模型 (部分模型下载需要 Token 进行验证)
# HuggingFace Token 在 Account -> Settings -> Access Tokens 中获取
HF_TOKEN = "" # HuggingFace Token
# ModelScope Token 在 首页 -> 访问令牌 -> SDK 令牌 中获取
MS_TOKEN = "" # ModelScope Token

# 用于上传模型的 HuggingFace 模型仓库的 ID, 当仓库不存在时则尝试新建一个
HF_REPO_ID = "username/reponame" # HuggingFace 仓库的 ID (格式: "用户名/仓库名")
HF_REPO_TYPE = "model" # HuggingFace 仓库的种类 (可选的类型为: model / dataset / space), 如果在 HuggingFace 新建的仓库为模型仓库则不需要修改
# HuggingFace 仓库类型和对应名称:
# model: 模型仓库
# dataset: 数据集仓库
# space: 在线运行空间仓库

# 用于上传模型的 ModelScope 模型仓库的 ID, 当仓库不存在时则尝试新建一个
MS_REPO_ID = "username/reponame" # ModelScope 仓库的 ID (格式: "用户名/仓库名")
MS_REPO_TYPE = "model" # ModelScope 仓库的种类 (model / dataset / space), 如果在 ModelScope 新建的仓库为模型仓库则不需要修改
# ModelScope 仓库类型和对应名称:
# model: 模型仓库
# dataset: 数据集仓库
# space: 创空间仓库

# 设置自动创建仓库时仓库的可见性, False 为私有仓库(不可见), True 为公有仓库(可见), 通常保持默认即可
HF_REPO_VISIBILITY = False # 设置新建的 HuggingFace 仓库可见性 (True / False)
MS_REPO_VISIBILITY = False # 设置新建的 ModelScope 仓库可见性 (True / False)

# Git 信息设置, 可以使用默认值
GIT_USER_EMAIL = "username@example.com" # Git 的邮箱
GIT_USER_NAME = "username" # Git 的用户名

##############################################################################

# 训练日志设置, 可使用 TensorBoard / WandB 记录训练日志, 使用 WandB 可远程查看实时训练日志
# 使用 WandB 需要填写 WANDB_TOKEN
# 如果 TensorBoard 和 WandB 同时使用, 可以改成 all
LOG_MODULE = "tensorboard" # 使用的日志记录工具 (tensorboard / wandb / all)

# WandB Token 设置
# WandB Token 可在 https://wandb.ai/authorize 中获取
WANDB_TOKEN = "" # WandB Token

##############################################################################

# 路径设置, 通常保持默认即可
INPUT_DATASET_PATH = "/kaggle/dataset" # 训练集保存的路径
OUTPUT_PATH = "/kaggle/working/model" # 训练时模型保存的路径
SD_MODEL_PATH = "/kaggle/sd-models" # 模型下载到的路径
KAGGLE_INPUT_PATH = "/kaggle/input" # Kaggle Input 的路径

##############################################################################

# 训练模型设置, 在安装时将会下载选择的模型
# 下面举个例子:
# SD_MODEL = [
#     ["https://huggingface.co/licyk/sd-model/resolve/main/sd_1.5/v1-5-pruned-emaonly.safetensors", 0],
#     ["https://huggingface.co/licyk/sd-model/resolve/main/sd_1.5/animefull-final-pruned.safetensors", 1],
#     ["https://huggingface.co/licyk/sd-model/resolve/main/sd_1.5/Counterfeit-V3.0_fp16.safetensors", 0],
#     ["https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/Illustrious-XL-v0.1.safetensors", 1, "Illustrious.safetensors"]
# ]
# 
# 在这个例子中, 第一个参数指定了模型的下载链接, 第二个参数设置了是否要下载这个模型, 当这个值为 1 时则下载该模型
# 第三个参数是可选参数, 用于指定下载到本地后的文件名称
# 
# 则上面的例子中
# https://huggingface.co/licyk/sd-model/resolve/main/sd_1.5/animefull-final-pruned.safetensors 和 
# https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/Illustrious-XL-v0.1.safetensors 下载链接所指的文件将被下载
# https://huggingface.co/licyk/sd-model/resolve/main/sd_1.5/animefull-final-pruned.safetensors 的文件下载到本地后名称为 animefull-final-pruned.safetensors
# 并且 https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/Illustrious-XL-v0.1.safetensors 所指的文件将被重命名为 Illustrious.safetensors

SD_MODEL = [
    # Stable Diffusion 模型
    ["https://huggingface.co/licyk/sd-model/resolve/main/sd_1.5/v1-5-pruned-emaonly.safetensors", 0],
    ["https://huggingface.co/licyk/sd-model/resolve/main/sd_1.5/animefull-final-pruned.safetensors", 1],
    ["https://huggingface.co/licyk/sd-model/resolve/main/sd_1.5/Counterfeit-V3.0_fp16.safetensors", 0],
    ["https://huggingface.co/licyk/sd-model/resolve/main/sd_1.5/cetusMix_Whalefall2.safetensors", 0],
    ["https://huggingface.co/licyk/sd-model/resolve/main/sd_1.5/cuteyukimixAdorable_neochapter3.safetensors", 0],
    ["https://huggingface.co/licyk/sd-model/resolve/main/sd_1.5/ekmix-pastel-fp16-no-ema.safetensors", 0],
    ["https://huggingface.co/licyk/sd-model/resolve/main/sd_1.5/ex2K_sse2.safetensors", 0],
    ["https://huggingface.co/licyk/sd-model/resolve/main/sd_1.5/kohakuV5_rev2.safetensors", 0],
    ["https://huggingface.co/licyk/sd-model/resolve/main/sd_1.5/meinamix_meinaV11.safetensors", 0],
    ["https://huggingface.co/licyk/sd-model/resolve/main/sd_1.5/oukaStar_10.safetensors", 0],
    ["https://huggingface.co/licyk/sd-model/resolve/main/sd_1.5/pastelMixStylizedAnime_pastelMixPrunedFP16.safetensors", 0],
    ["https://huggingface.co/licyk/sd-model/resolve/main/sd_1.5/rabbit_v6.safetensors", 0],
    ["https://huggingface.co/licyk/sd-model/resolve/main/sd_1.5/sweetSugarSyndrome_rev15.safetensors", 0],
    ["https://huggingface.co/licyk/sd-model/resolve/main/sd_1.5/AnythingV5Ink_ink.safetensors", 0],
    ["https://huggingface.co/licyk/sd-model/resolve/main/sd_1.5/bartstyledbBlueArchiveArtStyleFineTunedModel_v10.safetensors", 0],
    ["https://huggingface.co/licyk/sd-model/resolve/main/sd_1.5/meinapastel_v6Pastel.safetensors", 0],
    ["https://huggingface.co/licyk/sd-model/resolve/main/sd_1.5/qteamixQ_omegaFp16.safetensors", 0],
    ["https://huggingface.co/licyk/sd-model/resolve/main/sd_1.5/tmndMix_tmndMixSPRAINBOW.safetensors", 0],
    ["https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/sd_xl_base_1.0_0.9vae.safetensors", 0],
    ["https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/sd_xl_refiner_1.0_0.9vae.safetensors", 0],
    ["https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/sd_xl_turbo_1.0_fp16.safetensors", 0],
    ["https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/animagine-xl-3.0-base.safetensors", 0],
    ["https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/animagine-xl-3.0.safetensors", 0],
    ["https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/animagine-xl-3.1.safetensors", 0],
    ["https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/animagine-xl-4.0.safetensors", 0],
    ["https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/animagine-xl-4.0-opt.safetensors", 0],
    ["https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/holodayo-xl-2.1.safetensors", 0],
    ["https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/kivotos-xl-2.0.safetensors", 0],
    ["https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/clandestine-xl-1.0.safetensors", 0],
    ["https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/UrangDiffusion-1.1.safetensors", 0],
    ["https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/RaeDiffusion-XL-v2.safetensors", 0],
    ["https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/sd_xl_anime_V52.safetensors", 0],
    ["https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/kohaku-xl-delta-rev1.safetensors", 0],
    ["https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/kohakuXLEpsilon_rev1.safetensors", 0],
    ["https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/kohaku-xl-epsilon-rev2.safetensors", 0],
    ["https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/kohaku-xl-epsilon-rev3.safetensors", 0],
    ["https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/kohaku-xl-zeta.safetensors", 0],
    ["https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/starryXLV52_v52.safetensors", 0],
    ["https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/heartOfAppleXL_v20.safetensors", 0],
    ["https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/heartOfAppleXL_v30.safetensors", 0],
    ["https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/baxlBartstylexlBlueArchiveFlatCelluloid_xlv1.safetensors", 0],
    ["https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/baxlBlueArchiveFlatCelluloidStyle_xlv3.safetensors", 0],
    ["https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/sanaexlAnimeV10_v10.safetensors", 0],
    ["https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/sanaexlAnimeV10_v11.safetensors", 0],
    ["https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/SanaeXL-Anime-v1.2-aesthetic.safetensors", 0],
    ["https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/SanaeXL-Anime-v1.3-aesthetic.safetensors", 0],
    ["https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/Illustrious-XL-v0.1.safetensors", 1],
    ["https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/Illustrious-XL-v0.1-GUIDED.safetensors", 0],
    ["https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/Illustrious-XL-v1.0.safetensors", 0],
    ["https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/Illustrious-XL-v1.1.safetensors", 0],
    ["https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/Illustrious-XL-v2.0-stable.safetensors", 0],
    ["https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/jruTheJourneyRemains_v25XL.safetensors", 0],
    ["https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/PVCStyleModelMovable_illustriousxl10.safetensors", 0],
    ["https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/miaomiaoHarem_v15a.safetensors", 0],
    ["https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/waiNSFWIllustrious_v80.safetensors", 0],
    ["https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/tIllunai3_v4.safetensors", 0],
    ["https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/noobaiXLNAIXL_earlyAccessVersion.safetensors", 0],
    ["https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/noobaiXLNAIXL_epsilonPred05Version.safetensors", 0],
    ["https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/noobaiXLNAIXL_epsilonPred075.safetensors", 0],
    ["https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/noobaiXLNAIXL_epsilonPred077.safetensors", 0],
    ["https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/noobaiXLNAIXL_epsilonPred10Version.safetensors", 0],
    ["https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/noobaiXLNAIXL_epsilonPred11Version.safetensors", 0],
    ["https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/noobaiXLNAIXL_vPredTestVersion.safetensors", 0],
    ["https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/noobaiXLNAIXL_vPred05Version.safetensors", 0],
    ["https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/noobaiXLNAIXL_vPred06Version.safetensors", 0],
    ["https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/noobaiXLNAIXL_vPred065SVersion.safetensors", 0],
    ["https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/noobaiXLNAIXL_vPred075SVersion.safetensors", 0],
    ["https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/noobaiXLNAIXL_vPred09RVersion.safetensors", 0],
    ["https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/noobaiXLNAIXL_vPred10Version.safetensors", 1],
    ["https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/PVCStyleModelMovable_nbxl12.safetensors", 0],
    ["https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/PVCStyleModelMovable_nbxlVPredV10.safetensors", 0],
    ["https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/ponyDiffusionV6XL_v6StartWithThisOne.safetensors", 0],
    ["https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/pdForAnime_v20.safetensors", 0],
    ["https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/tPonynai3_v51WeightOptimized.safetensors", 0],
    ["https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/omegaPonyXLAnime_v20.safetensors", 0],
    ["https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/animeIllustDiffusion_v061.safetensors", 0],
    ["https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/artiwaifuDiffusion_v10.safetensors", 0],
    ["https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/artiwaifu-diffusion-v2.safetensors", 0],

    # VAE 模型
    ["https://huggingface.co/licyk/sd-vae/resolve/main/sd_1.5/vae-ft-ema-560000-ema-pruned.safetensors", 0],
    ["https://huggingface.co/licyk/sd-vae/resolve/main/sd_1.5/vae-ft-mse-840000-ema-pruned.safetensors", 1],
    ["https://huggingface.co/licyk/sd-vae/resolve/main/sdxl_1.0/sdxl_fp16_fix_vae.safetensors", 1],
]

##############################################################################
# 下面为初始化参数部分, 不需要修改
INSTALL_PARAMS = {
    "torch_ver": TORCH_VER or None,
    "xformers_ver": XFORMERS_VER or None,
    "git_branch": SD_SCRIPTS_BRANCH or None,
    "git_commit": SD_SCRIPTS_COMMIT or None,
    "model_path": SD_MODEL_PATH or None,
    "model_list": SD_MODEL,
    "use_uv": USE_UV,
    "pypi_index_mirror": PIP_INDEX_MIRROR or None,
    "pypi_extra_index_mirror": PIP_EXTRA_INDEX_MIRROR or None,
    # Kaggle 的环境暂不需要以下镜像源
    # "pypi_find_links_mirror": PIP_FIND_LINKS_MIRROR or None,
    # "github_mirror": GITHUB_MIRROR or None,
    # "huggingface_mirror": HUGGINGFACE_MIRROR or None,
    "pytorch_mirror": PYTORCH_MIRROR or None,
    "sd_scripts_repo": SD_SCRIPTS_REPO or None,
    "sd_scripts_requirment": SD_SCRIPTS_REQUIREMENT or None,
    "retry": RETRY,
    "huggingface_token": HF_TOKEN or None,
    "modelscope_token": MS_TOKEN or None,
    "wandb_token": WANDB_TOKEN or None,
    "git_username": GIT_USER_NAME or None,
    "git_email": GIT_USER_EMAIL or None,
    "check_avaliable_gpu": CHECK_AVALIABLE_GPU,
    "enable_tcmalloc": ENABLE_TCMALLOC,
    "enable_cuda_malloc": ENABLE_CUDA_MALLOC,
}
os.makedirs(WORKSPACE, exist_ok=True) # 创建工作路径
os.makedirs(OUTPUT_PATH, exist_ok=True) # 创建模型输出路径
os.makedirs(INPUT_DATASET_PATH, exist_ok=True) # 创建训练集路径
os.makedirs(SD_MODEL_PATH, exist_ok=True) # 创建模型下载路径
SD_SCRIPTS_PATH = os.path.join(WORKSPACE, WORKFOLDER) # sd-scripts 路径
logger.info("参数设置完成")

## 安装环境
安装环境和下载模型和训练集, 根据注释的说明进行修改  
3. [[← 上一个单元](#参数配置)|[下一个单元 →](#模型训练)]

In [ ]:
# 初始化部分参数并执行安装命令, 这一小部分不需要修改
from tqdm import tqdm
logger.info("开始安装 sd-scripts")
sd_scripts = SDScriptsManager(WORKSPACE, WORKFOLDER)
sd_scripts.install(**INSTALL_PARAMS)

# 将 KAGGLE_INPUT_PATH 内的文件移动到 INPUT_DATASET_PATH 指定的路径
if os.path.exists(KAGGLE_INPUT_PATH) and len(os.listdir(KAGGLE_INPUT_PATH)) > 0:
    logger.info("从 Kaggle Input 导入文件中")
    for i in tqdm(os.listdir(KAGGLE_INPUT_PATH), desc="[SD Scripts Manager] Kaggle Input 文件导入"):
        sd_scripts.copy_files(os.path.join(KAGGLE_INPUT_PATH, i), INPUT_DATASET_PATH)
# 在 Kaggle 界面右侧中有 Kaggle Input 的功能, 可用于导入训练集 / 模型
# 如果使用了 Kaggle Input 导入了训练集 / 模型, 则 KAGGLE_INPUT_PATH 中的所有文件将被复制到 INPUT_DATASET_PATH 中
# 即将 /kaggle/input 中的所有文件复制到 /kaggle/dataset 中
##########################################################################################
# 下方可自行编写命令
# 下方的命令示例可以根据自己的需求进行修改


##### 1. 关于运行环境 #####

# 如果需要安装某个软件包, 可以使用 %pip 命令
# 下面是几个使用例子:
# 1.
# %pip install lycoris-lora==2.1.0.post3 dadaptation==3.1
# 
# 这将安装 lycoris-lora==2.1.0.post3 和 dadaptation==3.1
# 
# 2.
# %pip uninstall tensorboard
# 
# 这将卸载 tensorboard


##########################################################################################


##### 2. 关于模型导入 #####

# 该 Kaggle 训练脚本支持 4 种方式导入模型, 如下:
# 1. 使用 Kaggle Input 导入
# 2. 使用模型下载链接导入
# 3. 从 HuggingFace 仓库导入
# 4. 从 ModelScope 仓库导入


### 2.1. 使用 Kaggle Input 导入 ###
# 在 Kaggle 右侧面板中, 点击 Notebook -> Input -> Upload -> New Model, 从此处导入模型


### 2.2 使用模型下载链接导入 ###
# 如果需要通过链接下载额外的模型, 可以使用 sd_scripts.get_model()
# 使用参数:
# sd_scripts.get_model(
#     url="model_url",                    # 模型下载链接
#     path=SD_MODEL_PATH,                 # 模型下载到本地的路径
#     filename="filename.safetensors",    # 模型的名称
#     retry=RETRY,                        # 重试下载的次数
# )
# 
# 下面是几个使用例子:
# 1.
# sd_scripts.get_model(
#     url="https://modelscope.cn/models/user/repo/resolve/master/your_model.safetensors",
#     path=SD_MODEL_PATH,
#     retry=RETRY,
# )
# 这将从 https://modelscope.cn/models/user/repo/resolve/master/your_model.safetensors 下载模型并保存到 SD_MODEL_PATH 中
# 
# sd_scripts.get_model(
#     url="https://modelscope.cn/models/user/repo/resolve/master/your_model.safetensors",
#     path=SD_MODEL_PATH,
#     filename="rename_model.safetensors",
#     retry=RETRY,
# )
# 这将从 https://modelscope.cn/models/user/repo/resolve/master/your_model.safetensors 下载模型并保存到 SD_MODEL_PATH 中, 并且重命名为 rename_model.safetensors


### 2.3. 从 HuggingFace 仓库导入 ###
# 如果需要从 HuggingFace 仓库下载模型, 可以使用 sd_scripts.repo.download_files_from_repo()
# 使用参数:
# sd_scripts.repo.download_files_from_repo(
#     api_type="huggingface",                 # 指定为 HuggingFace 的仓库
#     local_dir=SD_MODEL_PATH,                # 模型下载到本地的路径
#     repo_id="usename/repo_id",              # HuggingFace 仓库 ID
#     repo_type="model",                      # (可选参数) HuggingFace 仓库种类 (model / dataset / space)
#     folder="path/in/repo/file.safetensors", # (可选参数) 文件在 HuggingFace 仓库中的路径
#     retry=RETRY,                            # (可选参数) 重试下载的次数, 默认为 3
#     num_threads=DOWNLOAD_THREAD,            # (可选参数) 下载线程
# )
# 
# 例如要从 stabilityai/stable-diffusion-xl-base-1.0 (类型为 model) 下载 sd_xl_base_1.0_0.9vae.safetensors
# sd_scripts.repo.download_files_from_repo(
#     api_type="huggingface",
#     repo_id="stabilityai/stable-diffusion-xl-base-1.0",
#     repo_type="model",
#     folder="sd_xl_base_1.0_0.9vae.safetensors",
#     local_dir=SD_MODEL_PATH,
#     retry=RETRY,
#     num_threads=DOWNLOAD_THREAD,
# )
# 则上述的命令将会从 stabilityai/stable-diffusion-xl-base-1.0 下载 sd_xl_base_1.0_0.9vae.safetensors 模型
# 并将模型保存到 SD_MODEL_PATH 中
# 注意 folder 填的是文件在 HuggingFace 仓库中的路径, 如果上述例子中的文件在仓库的 checkpoint/sd_xl_base_1.0_0.9vae.safetensors 路径
# 则 folder 填的内容为 checkpoint/sd_xl_base_1.0_0.9vae.safetensors
#
# 模型保存的路径与 local_dir 和 folder 参数有关
# 对于上面的例子 local_dir 为 /kaggle/sd-models, folder 为 sd_xl_base_1.0_0.9vae.safetensors
# 则最终保存的路径为 /kaggle/sd-models/sd_xl_base_1.0_0.9vae.safetensors
# 
# 如果 folder 为 checkpoint/sd_xl_base_1.0_0.9vae.safetensors
# 则最终保存的路径为 /kaggle/sd-models/checkpoint/sd_xl_base_1.0_0.9vae.safetensors
# 
# folder 参数为可选参数, 即该参数可不指定, 在不指定的情况下将下载整个仓库中的文件
# 比如将上面的例子改成:
# sd_scripts.repo.download_files_from_repo(
#     api_type="huggingface",
#     repo_id="stabilityai/stable-diffusion-xl-base-1.0",
#     repo_type="model",
#     local_dir=SD_MODEL_PATH,
#     retry=RETRY,
#     num_threads=DOWNLOAD_THREAD,
# )
# 这时候将下载 stabilityai/stable-diffusion-xl-base-1.0 仓库中的所有文件
# 对于可选参数, 可以进行省略, 此时将使用该参数的默认值进行运行, 上面的例子就可以简化成下面的:
# sd_scripts.repo.download_files_from_repo(
#     api_type="huggingface",
#     repo_id="stabilityai/stable-diffusion-xl-base-1.0",
#     repo_type="model",
#     folder="sd_xl_base_1.0_0.9vae.safetensors",
#     local_dir=SD_MODEL_PATH,
# )
# 省略后仍然可以正常执行, 但对于一些重要的可选参数, 不推荐省略, 如 repo_type 参数
# 该参数用于指定仓库类型, 不指定时则默认认为仓库为 model 类型
# 若要下载的仓库为 dataset 类型, 不指定 repo_type 参数时默认就把仓库类型当做 model, 最终导致找不到要下载的仓库


### 2.4. 从 ModelScope 仓库导入 ###
# 如果需要从 ModelScope 仓库下载模型, 可以使用 sd_scripts.repo.download_files_from_repo()
# 使用方法和 **2.3. 从 HuggingFace 仓库导入** 部分的类似, 只需要指定 api_type="modelscope" 来指定使用 ModelScope 的仓库
# 使用参数:
# sd_scripts.repo.download_files_from_repo(
#     api_type="modelscope",                  # 指定为 ModelScope 的仓库
#     local_dir=SD_MODEL_PATH,                # 模型下载到本地的路径
#     repo_id="usename/repo_id",              # ModelScope 仓库 ID
#     repo_type="model",                      # (可选参数) ModelScope 仓库种类 (model / dataset / space)
#     folder="path/in/repo/file.safetensors", # (可选参数) 文件在 ModelScope 仓库中的路径
#     retry=RETRY,                            # (可选参数) 重试下载的次数, 默认为 3
#     num_threads=DOWNLOAD_THREAD,            # (可选参数) 下载线程
# )
# 
# 例如要从 stabilityai/stable-diffusion-xl-base-1.0 (类型为 model) 下载 sd_xl_base_1.0_0.9vae.safetensors
# sd_scripts.dataset.get_single_file_from_ms(
#     api_type="modelscope",
#     repo_id="stabilityai/stable-diffusion-xl-base-1.0",
#     repo_type="model",
#     folder="sd_xl_base_1.0_0.9vae.safetensors",
#     local_dir=SD_MODEL_PATH,
#     retry=RETRY,
#     num_threads=DOWNLOAD_THREAD,
# )
# 则上述的命令将会从 stabilityai/stable-diffusion-xl-base-1.0 下载 sd_xl_base_1.0_0.9vae.safetensors 模型
# 并将模型保存到 SD_MODEL_PATH 中



##########################################################################################


##### 3. 关于训练集导入 #####

# 该 Kaggle 训练脚本支持 4 种方式导入训练集, 如下:
# 1. 使用 Kaggle Input 导入
# 2. 使用训练集下载链接导入
# 3. 从 HuggingFace 仓库导入
# 4. 从 ModelScope 仓库导入


### 3.1. 使用 Kaggle Input 导入 ###
# 在 Kaggle 右侧面板中, 点击 Notebook -> Input -> Upload -> New Dataset, 从此处导入模型


### 3.2. 使用训练集下载链接导入 ###
# 如果将训练集压缩后保存在某个平台, 如 HuggingFace, ModelScope, 并且有下载链接
# 可以使用 sd_scripts.utils.download_archive_and_unpack() 函数下载训练集
# 使用参数:
# sd_scripts.utils.download_archive_and_unpack(
#     url="download_url",             # 训练集压缩包的下载链接
#     local_dir=INPUT_DATASET_PATH,   # 下载数据集到本地的路径
#     name="filename.zip",            # (可选参数) 将数据集压缩包进行重命名
#     retry=RETRY,                    # (可选参数) 重试下载的次数
# )
# 
# 该函数在下载训练集压缩包完成后将解压到指定的本地路径
# 压缩包格式仅支持 7z, zip, tar
# 
# 下面是几个使用的例子:
# 1.
# sd_scripts.utils.download_archive_and_unpack(
#     url="https://modelscope.cn/models/user/repo/resolve/master/data_1.7z",
#     local_dir=INPUT_DATASET_PATH,
#     retry=RETRY,
# )
# 这将从 https://modelscope.cn/models/user/repo/resolve/master/data_1.7z 下载训练集压缩包并解压到 INPUT_DATASET_PATH 中
# 
# 2.
# sd_scripts.utils.download_archive_and_unpack(
#     url="https://modelscope.cn/models/user/repo/resolve/master/data_1.7z",
#     local_dir=INPUT_DATASET_PATH,
#     name="training_dataset.7z",
#     retry=RETRY,
# )
# 这将从 https://modelscope.cn/models/user/repo/resolve/master/data_1.7z 下载训练集压缩包并重命名成 training_dataset.7z
# 再将 training_dataset.7z 中的文件解压到 INPUT_DATASET_PATH 中
# 
# 
# 训练集的要求:
# 需要将图片进行打标, 并调整训练集为指定的目结构, 例如:
# Nachoneko
#     └── 1_nachoneko
#             ├── [メロンブックス (よろず)]Melonbooks Girls Collection 2019 winter 麗.png
#             ├── [メロンブックス (よろず)]Melonbooks Girls Collection 2019 winter 麗.txt
#             ├── [メロンブックス (よろず)]Melonbooks Girls Collection 2020 spring 彩 (オリジナル).png
#             ├── [メロンブックス (よろず)]Melonbooks Girls Collection 2020 spring 彩 (オリジナル).txt
#             ├── 0(8).txt
#             ├── 0(8).webp
#             ├── 001_2.png
#             ├── 001_2.txt
#             ├── 0b1c8893-c9aa-49e5-8769-f90c4b6866f5.png
#             ├── 0b1c8893-c9aa-49e5-8769-f90c4b6866f5.txt
#             ├── 0d5149dd-3bc1-484f-8c1e-a1b94bab3be5.png
#             └── 0d5149dd-3bc1-484f-8c1e-a1b94bab3be5.txt
# 
# 在 Nachoneko 文件夹新建一个文件夹, 格式为 <数字>_<名称>, 如 1_nachoneko, 前面的数字代表这部分的训练集的重复次数, 1_nachoneko 文件夹内则放图片和打标文件
# 
# 训练集也可以分成多个部分组成, 例如:
# Nachoneko
#     ├── 1_nachoneko
#     │       ├── [メロンブックス (よろず)]Melonbooks Girls Collection 2019 winter 麗.png
#     │       ├── [メロンブックス (よろず)]Melonbooks Girls Collection 2019 winter 麗.txt
#     │       ├── [メロンブックス (よろず)]Melonbooks Girls Collection 2020 spring 彩 (オリジナル).png
#     │       └── [メロンブックス (よろず)]Melonbooks Girls Collection 2020 spring 彩 (オリジナル).txt
#     ├── 2_nachoneko
#     │       ├── 0(8).txt
#     │       ├── 0(8).webp
#     │       ├── 001_2.png
#     │       └── 001_2.txt
#     └── 4_nachoneko
#             ├── 0b1c8893-c9aa-49e5-8769-f90c4b6866f5.png
#             ├── 0b1c8893-c9aa-49e5-8769-f90c4b6866f5.txt
#             ├── 0d5149dd-3bc1-484f-8c1e-a1b94bab3be5.png
#             └── 0d5149dd-3bc1-484f-8c1e-a1b94bab3be5.txt
# 
# 处理好训练集并调整好目录结构后可以将 Nachoneko 文件夹进行压缩了, 使用 zip / 7z / tar 格式进行压缩
# 例如将上述的训练集压缩成 Nachoneko.7z, 此时需要检查一下压缩后在压缩包的目录结果是否和原来的一致(有些压缩软件在部分情况下会破坏原来的目录结构)
# 确认没有问题后将该训练集上传到网盘, 推荐使用 HuggingFace / ModelScope


### 3.3. 从 HuggingFace 仓库导入 ###
# 如果训练集保存在 HuggingFace, 可以使用 sd_scripts.repo.download_files_from_repo() 函数从 HuggingFace 下载数据集
# 使用方法和 **2.3. 从 HuggingFace 仓库导入** 部分类似, 部分说明可参考那部分的内容
# 使用格式:
# sd_scripts.repo.download_files_from_repo(
#     api_type="huggingface",                 # 指定为 HuggingFace 的仓库
#     local_dir=INPUT_DATASET_PATH,           # 下载数据集到哪个路径
#     repo_id="username/train_data",          # HuggingFace 仓库 ID
#     repo_type="dataset",                    # (可选参数) HuggingFace 仓库的类型 (model / dataset / space)
#     folder="folder_in_repo",                # (可选参数) 指定要从 HuggingFace 仓库里下载哪个文件夹的内容
#     retry=RETRY,                            # (可选参数) 重试下载的次数, 默认为 3
#     num_threads=DOWNLOAD_THREAD,            # (可选参数) 下载线程
# )
# 
# 比如在 HuggingFace 的仓库为 username/train_data, 仓库类型为 dataset
# 仓库的文件结构如下:
# ├── Nachoneko
# │   ├── 1_nachoneko
# │   │       ├── [メロンブックス (よろず)]Melonbooks Girls Collection 2019 winter 麗.png
# │   │       ├── [メロンブックス (よろず)]Melonbooks Girls Collection 2019 winter 麗.txt
# │   │       ├── [メロンブックス (よろず)]Melonbooks Girls Collection 2020 spring 彩 (オリジナル).png
# │   │       └── [メロンブックス (よろず)]Melonbooks Girls Collection 2020 spring 彩 (オリジナル).txt
# │   ├── 2_nachoneko
# │   │       ├── 0(8).txt
# │   │       ├── 0(8).webp
# │   │       ├── 001_2.png
# │   │       └── 001_2.txt
# │   └── 4_nachoneko
# │           ├── 0b1c8893-c9aa-49e5-8769-f90c4b6866f5.png
# │           ├── 0b1c8893-c9aa-49e5-8769-f90c4b6866f5.txt
# │           ├── 0d5149dd-3bc1-484f-8c1e-a1b94bab3be5.png
# │           └── 0d5149dd-3bc1-484f-8c1e-a1b94bab3be5.txt
# └ aaaki
#   ├── 1_aaaki
#   │   ├── 1.png
#   │   ├── 1.txt
#   │   ├── 11.png
#   │   ├── 11.txt
#   │   ├── 12.png
#   │   └── 12.txt
#   └── 3_aaaki
#       ├── 14.png
#       ├── 14.txt
#       ├── 16.png
#       └── 16.txt
#
# 此时想要下载这个仓库中的 Nachoneko 文件夹的内容, 则下载命令为
# sd_scripts.repo.download_files_from_repo(
#     api_type="huggingface",
#     local_dir=INPUT_DATASET_PATH,
#     repo_id="username/train_data",
#     repo_type="dataset",
#     folder="Nachoneko",
#     retry=RETRY,
#     num_threads=DOWNLOAD_THREAD,
# )
# 
# 如果想下载整个仓库, 则移除 folder 参数, 命令修改为
# sd_scripts.repo.download_files_from_repo(
#     api_type="huggingface",
#     local_dir=INPUT_DATASET_PATH,
#     repo_id="username/train_data",
#     repo_type="dataset",
#     retry=RETRY,
#     num_threads=DOWNLOAD_THREAD,
# )



# 4. 从 ModelScope 仓库导入
# 如果训练集保存在 ModelScope, 可以使用 sd_scripts.repo.download_files_from_repo() 函数从 ModelScope 下载数据集
# 使用方法可参考 **3.2. 使用训练集下载链接导入** 部分的说明
# 使用格式:
# sd_scripts.repo.download_files_from_repo(
#     api_type="modelscope",          # 指定为 ModelScope 的仓库
#     local_dir=INPUT_DATASET_PATH,   # 下载数据集到哪个路径
#     repo_id="usename/repo_id",      # ModelScope 仓库 ID
#     repo_type="dataset",            # (可选参数) ModelScope 仓库的类型 (model / dataset / space)
#     folder="folder_in_repo",        # (可选参数) 指定要从 ModelScope 仓库里下载哪个文件夹的内容
#     retry=RETRY,                    # (可选参数) 重试下载的次数, 默认为 3
#     num_threads=DOWNLOAD_THREAD,    # (可选参数) 下载线程
# )
# 
# 比如在 ModelScope 的仓库为 username/train_data, 仓库类型为 dataset
# 仓库的文件结构如下:
# ├── Nachoneko
# │   ├── 1_nachoneko
# │   │       ├── [メロンブックス (よろず)]Melonbooks Girls Collection 2019 winter 麗.png
# │   │       ├── [メロンブックス (よろず)]Melonbooks Girls Collection 2019 winter 麗.txt
# │   │       ├── [メロンブックス (よろず)]Melonbooks Girls Collection 2020 spring 彩 (オリジナル).png
# │   │       └── [メロンブックス (よろず)]Melonbooks Girls Collection 2020 spring 彩 (オリジナル).txt
# │   ├── 2_nachoneko
# │   │       ├── 0(8).txt
# │   │       ├── 0(8).webp
# │   │       ├── 001_2.png
# │   │       └── 001_2.txt
# │   └── 4_nachoneko
# │           ├── 0b1c8893-c9aa-49e5-8769-f90c4b6866f5.png
# │           ├── 0b1c8893-c9aa-49e5-8769-f90c4b6866f5.txt
# │           ├── 0d5149dd-3bc1-484f-8c1e-a1b94bab3be5.png
# │           └── 0d5149dd-3bc1-484f-8c1e-a1b94bab3be5.txt
# └ aaaki
#   ├── 1_aaaki
#   │   ├── 1.png
#   │   ├── 1.txt
#   │   ├── 11.png
#   │   ├── 11.txt
#   │   ├── 12.png
#   │   └── 12.txt
#   └── 3_aaaki
#       ├── 14.png
#       ├── 14.txt
#       ├── 16.png
#       └── 16.txt
#
# 此时想要下载这个仓库中的 Nachoneko 文件夹的内容, 则下载命令为
# sd_scripts.repo.download_files_from_repo(
#     api_type="modelscope",
#     local_dir=INPUT_DATASET_PATH,
#     repo_id="username/train_data",
#     repo_type="dataset",
#     folder="Nachoneko",
# )
# 
# 如果想下载整个仓库, 则移除 folder 参数, 命令修改为
# sd_scripts.repo.download_files_from_repo(
#     api_type="modelscope",
#     local_dir=INPUT_DATASET_PATH,
#     repo_id="username/train_data",
#     repo_type="dataset",
# )



# 下载训练集的技巧
# 如果有个 character_aaaki 训练集上传到 HuggingFace 上时结构如下：
# 
#
# HuggingFace_Repo (licyk/sd_training_dataset)
# ├── character_aaaki
# │   ├── 1_aaaki
# │   │   ├── 1.png
# │   │   ├── 1.txt
# │   │   ├── 3.png
# │   │   └── 3.txt
# │   └── 2_aaaki
# │       ├── 4.png
# │       └── 4.txt
# ├── character_robin
# │   └── 1_xxx
# │       ├── 11.png
# │       └── 11.txt
# └── style_pvc
#     └── 5_aaa
#         ├── test.png
#         └── test.txt
#
# 
# 可能有时候不想为训练集中每个子训练集设置不同的重复次数，又不想上传的时候再多套一层文件夹，就把训练集结构调整成了下面的：
# 
#
# HuggingFace_Repo (licyk/sd_training_dataset)
# ├── character_aaaki
# │   ├── 1.png
# │   ├── 1.txt
# │   ├── 3.png
# │   ├── 3.txt
# │   ├── 4.png
# │   └── 4.txt
# ├── character_robin
# │   └── 1_xxx
# │       ├── 11.png
# │       └── 11.txt
# └── style_pvc
#     └── 5_aaa
#         ├── test.png
#         └── test.txt
#
# 
# 此时这个状态的训练集是缺少子训练集和重复次数的，如果直接使用 sd_scripts.repo.download_files_from_repo() 去下载训练集并用于训练将会导致报错
# 不过可以自己再编写一个函数对 sd_scripts.repo.download_files_from_repo() 函数再次封装，自动加上子训练集并设置重复次数
# 
#
# def make_dataset(
#     local_dir: str | Path,
#     repo_id: str,
#     repo_type: str,
#     repeat: int,
#     folder: str,
# ) -> None:
#     import os
#     import shutil
#     origin_dataset_path = os.path.join(local_dir, folder)
#     tmp_dataset_path = os.path.join(local_dir, f"{repeat}_{folder}")
#     new_dataset_path = os.path.join(origin_dataset_path, f"{repeat}_{folder}")
#     sd_scripts.repo.download_files_from_repo(
#         api_type="huggingface",
#         local_dir=local_dir,
#         repo_id=repo_id,
#         repo_type=repo_type,
#         folder=folder,
#     )
#     if os.path.exists(origin_dataset_path):
#         logger.info("设置 %s 训练集的重复次数为 %s", folder, repeat)
#         shutil.move(origin_dataset_path, tmp_dataset_path)
#         shutil.move(tmp_dataset_path, new_dataset_path)
#     else:
#         logger.error("从 %s 下载 %s 失败", repo_id, folder)
#
# 
# 编写好后，可以去调用这个函数
# 
#
# make_dataset(
#     local_dir=INPUT_DATASET_PATH,
#     repo_id="licyk/sd_training_dataset",
#     repo_type="dataset",
#     repeat=3,
#     folder="character_aaaki",
# )
#
# 
# 该函数将会把 character_aaaki 训练集下载到 {INPUT_DATASET_PATH} 中，即 /kaggle/dataset
# 文件夹名称为 character_aaaki，并且 character_aaaki 文件夹内继续创建了一个子文件夹作为子训练集，根据 repeat=3 将子训练集的重复次数设置为 3


##########################################################################################
logger.info("sd-scripts 安装完成")

## 模型训练
需自行编写命令，下方有可参考的例子  
4. [[← 上一个单元](#安装环境)|[下一个单元 →](#模型上传)]

In [ ]:
logger.info("进入 sd-scripts 目录")
os.chdir(SD_SCRIPTS_PATH)
print("=" * 50)
logger.info("训练集目录中的文件列表")
if os.path.exists(INPUT_DATASET_PATH):
    print(f"训练集路径: {INPUT_DATASET_PATH}")
    for i in os.listdir(INPUT_DATASET_PATH):
        print(f":: {i}")
print("=" * 50)
logger.info("模型目录中的文件列表")
if os.path.exists(SD_MODEL_PATH):
    print(f"模型路径: {SD_MODEL_PATH}")
    for i in os.listdir(SD_MODEL_PATH):
        print(f":: {i}")
print("=" * 50)
logger.info("使用 sd-scripts 进行模型训练")
##########################################################################################
# 1.
# 运行前需要根据自己的需求更改参数
# 
# 训练参数的设置可参考：
# https://rentry.org/59xed3
# https://github.com/kohya-ss/sd-scripts?tab=readme-ov-file#links-to-usage-documentation
# https://github.com/bmaltais/kohya_ss/wiki/LoRA-training-parameters
# https://sd-moadel-doc.maozi.io
# 
# 
# 2.
# 下方被注释的代码选择后使用 Ctrl + / 取消注释
# 
# 
# 3.
# 训练使用的底模会被下载到 SD_MODEL_PATH, 即 /kaggle/sd-models
# 填写底模路径时一般可以通过 --pretrained_model_name_or_path="{SD_MODEL_PATH}/base_model.safetensors" 指定
# 如果需要外挂 VAE 模型可以通过 --vae="{SD_MODEL_PATH}/vae.safetensors" 指定
# 
# 通过 Kaggle Inout 导入的训练集保存在 KAGGLE_INPUT_PATH, 即 /kaggle/input, 运行该笔记时将会把训练集复制进 INPUT_DATASET_PATH, 即 /kaggle/dataset
# 该路径可通过 INPUT_DATASET_PATH 调整
# 如果使用 sd_scripts.dataset.get_dataset() 函数下载训练集, 数据集一般会解压到 INPUT_DATASET_PATH, 这取决于函数第一个参数传入的路径
# 训练集的路径通常要这种结构
# $ tree /kaggle
# kaggle
# └── dataset
#     └── Nachoneko
#         └── 1_gan_cheng
#             ├── [メロンブックス (よろず)]Melonbooks Girls Collection 2019 winter 麗.png
#             ├── [メロンブックス (よろず)]Melonbooks Girls Collection 2019 winter 麗.txt
#             ├── [メロンブックス (よろず)]Melonbooks Girls Collection 2020 spring 彩 (オリジナル).png
#             ├── [メロンブックス (よろず)]Melonbooks Girls Collection 2020 spring 彩 (オリジナル).txt
#             ├── 0(8).txt
#             ├── 0(8).webp
#             ├── 001_2.png
#             ├── 001_2.txt
#             ├── 0b1c8893-c9aa-49e5-8769-f90c4b6866f5.png
#             ├── 0b1c8893-c9aa-49e5-8769-f90c4b6866f5.txt
#             ├── 0d5149dd-3bc1-484f-8c1e-a1b94bab3be5.png
#             └── 0d5149dd-3bc1-484f-8c1e-a1b94bab3be5.txt
# 4 directories, 12 files
# 在填写训练集路径时, 应使用 --train_data_dir="{INPUT_DATASET_PATH}/Nachoneko"
# 
# 模型保存的路径通常用 --output_dir="{OUTPUT_PATH}" 指定, 如 --output_dir="{OUTPUT_PATH}/Nachoneko", OUTPUT_PATH 默认设置为 /kaggle/working/model
# 在 Kaggle 的 Output 中可以看到保存的模型, 前提是使用 Kaggle 的 Save Version 运行 Kaggle
# OUTPUT_PATH 也指定了保存模型到 HuggingFace / ModelScope 的功能的上传路径
# 
# --output_name 用于指定保存的模型名字, 如 --output_name="Nachoneko"
# 
# 
# 4.
# Kaggle 的实例最长可运行 12 h, 要注意训练时长不要超过 12 h, 否则将导致训练被意外中断, 并且最后的模型保存功能将不会得到运行
# 如果需要在模型被保存后立即上传到 HuggingFace 进行保存, 可使用启动参数为 sd-scripts 设置自动保存, 具体可阅读 sd-scripts 的帮助信息
# 使用 python train_network.py -h 命令可查询可使用的启动参数, 命令中的 train_network.py 可替换成 sdxl_train_network.py 等
# 
# 
# 5.
# 训练命令的开头为英文的感叹号, 也就是 !, 后面就是 Shell Script 风格的命令
# 每行的最后为反斜杠用于换行, 也就是用 \ 来换行, 并且反斜杠的后面不允许有其他符号, 比如空格等
# 训练命令的每一行之间不能有任何换行空出来, 最后一行不需要反斜杠, 因为最后一行的下一行已经没有训练参数
# 
# 
# 6.
# 如果训练参数是 toml 格式的, 比如从 Akegarasu/lora-scripts 训练器复制来的训练参数
# 可以转换成对应的训练命令中的参数
# 下面列举几种转换例子:
# 
# (1)
# toml 格式:
# pretrained_model_name_or_path = "{SD_MODEL_PATH}/Illustrious-XL-v0.1.safetensors"
# 训练命令格式:
# --pretrained_model_name_or_path="{SD_MODEL_PATH}/Illustrious-XL-v0.1.safetensors"
# 
# (2)
# toml 格式:
# unet_lr = 0.0001
# 训练命令格式:
# --unet_lr=0.0001
# 
# (3)
# toml 格式:
# network_args = [
#     "conv_dim=100000",
#     "conv_alpha=100000",
#     "algo=lokr",
#     "dropout=0",
#     "factor=8",
#     "train_norm=True",
#     "preset=full",
# ]
# 训练命令格式:
# --network_args \
#     conv_dim=100000 \
#     conv_alpha=100000 \
#     algo=lokr \
#     dropout=0 \
#     factor=8 \
#     train_norm=True \
#     preset=full \
# 
# (4)
# toml 格式:
# enable_bucket = true
# 训练命令格式:
# --enable_bucket
# 
# (5)
# toml 格式:
# lowram = false
# 训练命令格式:
# 无对应的训练命令, 也就是不需要填, 因为这个参数的值为 false, 也就是无对应的参数, 如果值为 true, 则对应训练命令中的 --lowram
# 
# 可以根据这个例子去转换 toml 格式的训练参数成训练命令的格式
# 
# 
# 7.
# 如果需要 toml 格式的配置文件来配置训练参数可以使用下面的代码来保存 toml 格式的训练参数
# 
# toml_file_path = os.path.join(WORKSPACE, "train_config.toml")
# toml_content = f"""
# 这里使用 toml 格式编写训练参数, 
# 还可以结合 Python F-Strings 的用法使用前面配置好的变量
# Python F-Strings 的说明: https://docs.python.org/zh-cn/3.13/reference/lexical_analysis.html#f-strings
# toml 的语法可参考: https://toml.io/cn/v1.0.0
# 下面展示训练命令里参数对应的 toml 格式转换
# 
# 
# pretrained_model_name_or_path = "{SD_MODEL_PATH}/Illustrious-XL-v0.1.safetensors"
# 对应训练命令中的 --pretrained_model_name_or_path="{SD_MODEL_PATH}/Illustrious-XL-v0.1.safetensors"
# 
# unet_lr = 0.0001
# 对应训练命令中的 --unet_lr=0.0001
# 
# network_args = [
#     "conv_dim=100000",
#     "conv_alpha=100000",
#     "algo=lokr",
#     "dropout=0",
#     "factor=8",
#     "train_norm=True",
#     "preset=full",
# ]
# 对应下面训练命令中的
# --network_args \
#     conv_dim=100000 \
#     conv_alpha=100000 \
#     algo=lokr \
#     dropout=0 \
#     factor=8 \
#     train_norm=True \
#     preset=full \
# 
# enable_bucket = true
# 对应训练命令中的 --enable_bucket
# 
# lowram = false
# 这个参数的值为 false, 也就是无对应的参数, 如果值为 true, 则对应训练命令中的 --lowram
# """.strip()
# if not os.path.exists(os.path.dirname(toml_file_path)):
#     os.makedirs(toml_file_path, exist_ok=True)
# with open(toml_file_path, "w", encoding="utf8") as file:
#     file.write(toml_content)
# 
# 使用上面的代码将会把训练参数的 toml 配置文件保存在 toml_file_path 路径中, 也就是 {WORKSPACE}/train_config.toml, 即 /kaggle/train_config.toml
# 而原来的训练命令无需再写上训练参数, 只需指定该训练配置文件的路径即可
# 使用 --config_file="{WORKSPACE}/train_config.toml" 来指定
# 
# 
# 8. 
# 如果要查看 sd-script 的命令行参数, 可以加上 -h 后再运行, 此时 sd-script 将显示所有可用的参数
# 
# 
# 9.
# 下方提供了一些训练参数, 可以直接使用, 使用时取消注释后根据需求修改部分参数即可
# 
#              .,@@@@@@@@@].                                          ./@[`....`[\\.                 
#             //\`..  . ...,\@].       .,]]]/O@@@@@@@@\]...       .,]//............\@`               
#           .O`........ .......\\.]]@@@@@@@@[..........,[@@@@\`.*/....=^............/@@`             
#          .O........    .......@@/@@@/`.....               . ,\@\....\`............O@`@             
#          =^...`....          .O@@`.........            .........\@`...[`.,@`....,@^/.@^            
#         .OO`..\....          =/..... ......            ..[@]....,\@@]]]].@@]`..//..@=\^            
#          @O/@`,............=O/......    ...   ....       ...\\.....,@@@`=\@\@@[...=O`/^.           
#          @@\.,@]..]]//[,/@^O=@.............   .\@^...........,@`.....\@@/*\o*O@\.=/.@`             
#          ,@/O`...[OOO`.,@O,\/....././\^....   ..@O` ..\`.......=\.....=\\@@@@@/\@@//               
#            ,@`\].......O^o,/.....@`/=^.....,\...,@^ ...=\...    =\.....,@,@@@@[/@@@/               
#            ..,\@\]]]]O/@.*@.....=^/\^......=@....\O..^..@@`..  ..\@.....,@.\@@\[[O`                
#                .*=@@\@@^.O^...../o^O.......O=^...=@..@..\.\\.   . @@`....,@.\@@@@`                 
# .              ..=@O^=@`,@ .....@@=`......=^.O....@..@^.=^.=@.....=@@.....,\.\@@@@.                
#                .,@@`,O@./^.....=@O/......./^.O... \`.=^.=^..=@...  O=\.....=^.\@@@@`.              
#                ./@`.=^@=@......=@O`....@,/@@.=^...=^.=^.=^.[[\@....=^\^.....@@.\@@@@`              
#               .,@^. @^O/@......=@O.]O`OO.=`\^.....,^.=@.=^....=@...=\.O.....@^\`@@/`               
#                =@ .=@..@^ .....=@/.../@^,/.=@......* =@.=^.....=\..=@`=^....=^ \/\                 
#                /^..=@.,@^ /`...=@.../O@.O...@........O=^=` ,`...@^.=@\=^..] =@..@O`.               
#               ,@...@/.=@. @^...=@../@\@/OOO.=^......=^,O@[.]]@\]/@ =^@`O..O.=@^ =@^                
#               =^...@@.=@..O\....@ //.O@O@@]..@....../^.OO@@@[[@@@@\/^@^O .O.=@@ .@^                
#               @^..=@@.,@.=^@....@@\@@@[[[[[[[\^@^..,/..O..,@@@\..=@@//OO..O./^@. =@                
#               @...=^@^.@.=^=^...=@@`/@@@@@`...*O\..@...[.=@`,@@@`.@`=^@=`.O.@.@. =@.               
#               @^..O.@^ \^=@,\..=@@ @\,@@/@@`..=^..@`.....@@\@@/@@...O.@=^,O=^.@^./@                
#               @@..O.=@.,\=@^O..=`\/@^/@@OO@^..,`,O`.. .. @@/@@\@@..=`=@../^O..@^/O^                
#              .=@^ @..@`=@o@@=^..,.O@@/@@oO@........... ...@^.\/@..=^=@/ =@O. .@\@`.                
#               .@@.@^.@^@^\@@O^..=^,O@@*.................  .......=^/@@^=@@^ .=@`.                  
#               .=O@O^,@^=^.\@^o..=/\,\ .....   .... .....    ...]@O`=@O/@@^   =@                    
#                .=O@O/^==@@`O@O^.=@.\`\`....      .  ........ ......//@.@`.   =^                    
#                  ,O@@^.O..\@@@^.=@...[O@`..      .  ........ .....//.@@,\   .@^                    
#                   .@/@@@]../@@^..@@\........     ..,/`=@/@`.....,@^..=^ =` .=@.                    
#                   =/..O... @^=\. O@@@@\....... ...//.,@..@O].,/@`=\..=@..@..@^                     
#                  ,/..O....=/=/@ .=@@@@@@@@].....//.,O@`.//.@@@@@..@^..@`.=\/O`.                    
#                 ,@../`....O=/.@...@@@@@@@@@@@@@/../\@..//.=@@@@\. @@..=\..@^/.                     
#                ,@`.=`....=@/..@...\@@@@@@@@.. O..,@/..=@ .O=@@@^. @/@..@^..@`.                     
#               ,@`.=^....,@/..=O^..,@@.[@@@@,@]@../^..=@../^=@@@...@.,\.=@`..@`                     
#              ,@..=/.   .@/...O=@...@@....,@@...,[@\.,@`..@..@@/..,@..,\.@@...@..                   
#              @`.,/.....@/...=`O@^..,@...../`.......,\@]..@..O@\]]/\`..,\,@^..,@`                   
#             =^..@...../@...,^/O@@...=@`...@............\@` /`,\,@`.=\.,\@=@`..,@.                  
#            ,@../`....@\`/@``,`]/@^...O,\/\@]..............\\..=@`\\ ,@@@@@@@....@`.                
#            O`.=^....@^O@^.@@@@@@@\....\@^=@@@@@\] ..........,@`\@\.@`=@@@@@@\....@`..              
#           =/..O...,@O/@^.@@@@@@@@@`...=/.@@@@@@@@@@@].........,@@/@`,@/@@@@O\^....@`..             
#           /^.O.../@O^@^./@@@@@@@@@\...@`=@@@@@@@@@@@@@\.......@`=\//@`,@@@@@.@`....\`...           
#         .,@.=^../`@@\@.=@@@@@@@@@@@^.=@.@@@@@@@@@@@@@@@@@`...@` =\\==`@`\@@@^.@.....\^...          
#       ../\^,@.,/.=@/=/,@@@@@@@@@@@@\ @^=@@@@@@@@@@@@@@@@@@@]@@@@@@@@@o\@`.@@\.,@.....\\...         
#       =/.@^@`/`..@@^@^=@@@@@@@@@@@@@\@.@@@@@@@@@@@@@@@@@@@@@@@@O@@@@@\/.,/.@@\.,@.....,@`..        
#     ,O`.,@=@/...=@@.@^O@@@@@@@@@@@@@@^=@@@@@@@@@@@@@@@@@@@@@@@@@@O@@@^ /@@.,@/@`.@`.....\\...      
#  ..=^...=^@^....@OO,@^/@@@@@@@@@@@@\@.@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@\@@@@@`=\.\^.\`.....,@`..    
# 
# 炼丹就不存在什么万能参数的, 可能下面给的参数里有的参数训练出来的效果很好, 有的效果就一般
# 训练参数还看训练集呢, 同一套参数在不同训练集上效果都不一样, 可能好可能坏 (唉, 被这训练参数折磨过好多次)
# 虽然说一直用同个效果不错的参数可能不会出现特别坏的结果吧
# 还有好的训练集远比好的训练参数更重要
# 好的训练集真的, 真的, 真的非常重要
# 再好的参数, 训练集烂也救不回来
# 
# 
# 10.
# 建议先改改训练集路径的参数就开始训练, 跑通训练了再试着改其他参数
# 还有我编写的训练参数不一定是最好的, 所以需要自己去摸索这些训练参数是什么作用的, 再去修改
# 其实有些参数我自己也调不明白, 但是很多时候跑出来效果还不错
# 为什么效果好, 分からない, 这东西像个黑盒, 有时候就觉得神奇呢
##########################################################################################


# 使用 lokr 算法训练 XL 画风 LoRA, 使用多卡进行训练
# 该参数也可以用于人物 LoRA 训练
# 
# 在训练多画风 LoRA 或者人物 LoRA 时, 通常会打上触发词
# 当使用了 --network_train_unet_only 后, Text Encoder 虽然不会训练, 但并不影响将触发词训练进 LoRA 模型中
# 并且不训练 Text Encoder 避免 Text Encoder 被炼烂(Text Encoder 比较容易被炼烂)
#
# 这个参数在 animagine-xl-3.1.safetensors 测试, 大概在 30 ~ 40 Epoch 有比较好的效果 (在 36 Epoch 出好效果的概率比较高)
#
# !python -m accelerate.commands.launch \
#     --num_cpu_threads_per_process=1 \
#     --multi_gpu \
#     --num_processes=2 \
#     "{SD_SCRIPTS_PATH}/sdxl_train_network.py" \
#     --pretrained_model_name_or_path="{SD_MODEL_PATH}/animagine-xl-3.1.safetensors" \
#     --vae="{SD_MODEL_PATH}/sdxl_fp16_fix_vae.safetensors" \
#     --train_data_dir="{INPUT_DATASET_PATH}/Nachoneko" \
#     --output_name="Nachoneko_2" \
#     --output_dir="{OUTPUT_PATH}/Nachoneko" \
#     --wandb_run_name="Nachoneko" \
#     --log_tracker_name="lora-Nachoneko" \
#     --prior_loss_weight=1 \
#     --resolution="1024,1024" \
#     --enable_bucket \
#     --min_bucket_reso=256 \
#     --max_bucket_reso=4096 \
#     --bucket_reso_steps=64 \
#     --save_model_as="safetensors" \
#     --save_precision="fp16" \
#     --save_every_n_epochs=1 \
#     --max_train_epochs=50 \
#     --train_batch_size=6 \
#     --gradient_checkpointing \
#     --network_train_unet_only \
#     --learning_rate=0.0001 \
#     --unet_lr=0.0001 \
#     --text_encoder_lr=0.00001 \
#     --lr_scheduler="cosine_with_restarts" \
#     --lr_warmup_steps=0 \
#     --lr_scheduler_num_cycles=1 \
#     --optimizer_type="Lion8bit" \
#     --network_module="lycoris.kohya" \
#     --network_dim=100000 \
#     --network_alpha=100000 \
#     --network_args \
#         conv_dim=100000 \
#         conv_alpha=100000 \
#         algo=lokr \
#         dropout=0 \
#         factor=8 \
#         train_norm=True \
#     --log_with="{LOG_MODULE}" \
#     --logging_dir="{OUTPUT_PATH}/logs" \
#     --caption_extension=".txt" \
#     --shuffle_caption \
#     --keep_tokens=0 \
#     --max_token_length=225 \
#     --seed=1337 \
#     --mixed_precision="fp16" \
#     --xformers \
#     --cache_latents \
#     --cache_latents_to_disk \
#     --persistent_data_loader_workers \
#     --vae_batch_size=4 \
#     --full_fp16


# 使用 lokr 算法训练 XL 画风 LoRA, 使用多卡进行训练
# 该参数也可以用于人物 LoRA 训练
# 
# 在训练多画风 LoRA 或者人物 LoRA 时, 通常会打上触发词
# 当使用了 --network_train_unet_only 后, Text Encoder 虽然不会训练, 但并不影响将触发词训练进 LoRA 模型中
# 并且不训练 Text Encoder 避免 Text Encoder 被炼烂(Text Encoder 比较容易被炼烂)
#
# 这个参数是在 Illustrious-XL-v0.1.safetensors 模型上测出来的, 大概在 32 Epoch 左右有比较好的效果
# 用 animagine-xl-3.1.safetensors 那套参数也有不错的效果, 只是学习率比这套低了点, 学得慢一点
#
# !python -m accelerate.commands.launch \
#     --num_cpu_threads_per_process=1 \
#     --multi_gpu \
#     --num_processes=2 \
#     "{SD_SCRIPTS_PATH}/sdxl_train_network.py" \
#     --pretrained_model_name_or_path="{SD_MODEL_PATH}/Illustrious-XL-v0.1.safetensors" \
#     --vae="{SD_MODEL_PATH}/sdxl_fp16_fix_vae.safetensors" \
#     --train_data_dir="{INPUT_DATASET_PATH}/Nachoneko" \
#     --output_name="Nachoneko_2" \
#     --output_dir="{OUTPUT_PATH}/Nachoneko" \
#     --wandb_run_name="Nachoneko" \
#     --log_tracker_name="lora-Nachoneko" \
#     --prior_loss_weight=1 \
#     --resolution="1024,1024" \
#     --enable_bucket \
#     --min_bucket_reso=256 \
#     --max_bucket_reso=4096 \
#     --bucket_reso_steps=64 \
#     --save_model_as="safetensors" \
#     --save_precision="fp16" \
#     --save_every_n_epochs=1 \
#     --max_train_epochs=40 \
#     --train_batch_size=6 \
#     --gradient_checkpointing \
#     --network_train_unet_only \
#     --learning_rate=0.00012 \
#     --unet_lr=0.00012 \
#     --text_encoder_lr=0.00001 \
#     --lr_scheduler="cosine_with_restarts" \
#     --lr_warmup_steps=0 \
#     --lr_scheduler_num_cycles=1 \
#     --optimizer_type="Lion8bit" \
#     --network_module="lycoris.kohya" \
#     --network_dim=100000 \
#     --network_alpha=100000 \
#     --network_args \
#         conv_dim=100000 \
#         conv_alpha=100000 \
#         algo=lokr \
#         dropout=0 \
#         factor=8 \
#         train_norm=True \
#     --log_with="{LOG_MODULE}" \
#     --logging_dir="{OUTPUT_PATH}/logs" \
#     --caption_extension=".txt" \
#     --shuffle_caption \
#     --keep_tokens=0 \
#     --max_token_length=225 \
#     --seed=1337 \
#     --mixed_precision="fp16" \
#     --xformers \
#     --cache_latents \
#     --cache_latents_to_disk \
#     --persistent_data_loader_workers \
#     --vae_batch_size=4 \
#     --full_fp16


# 使用 lokr 算法训练 XL 画风 LoRA, 使用多卡进行训练
# 该参数也可以用于人物 LoRA 训练
# 
# 在训练多画风 LoRA 或者人物 LoRA 时, 通常会打上触发词
# 当使用了 --network_train_unet_only 后, Text Encoder 虽然不会训练, 但并不影响将触发词训练进 LoRA 模型中
# 并且不训练 Text Encoder 避免 Text Encoder 被炼烂(Text Encoder 比较容易被炼烂)
#
# 这个参数是在 Illustrious-XL-v0.1.safetensors 模型上测出来的, 大概在 32 Epoch 左右有比较好的效果
# 用 animagine-xl-3.1.safetensors 那套参数也有不错的效果, 只是学习率比这套低了点, 学得慢一点
# 学习率调度器从 cosine_with_restarts 换成 constant_with_warmup, 此时学习率靠优化器(Lion8bit)进行调度
# 拟合速度会更快
# constant_with_warmup 用在大规模的训练上比较好, 但用在小规模训练也有不错的效果
# 如果训练集的图比较少, 重复的图较多, 重复次数较高, 可能容易造成过拟合
# 
# !python -m accelerate.commands.launch \
#     --num_cpu_threads_per_process=1 \
#     --multi_gpu \
#     --num_processes=2 \
#     "{SD_SCRIPTS_PATH}/sdxl_train_network.py" \
#     --pretrained_model_name_or_path="{SD_MODEL_PATH}/Illustrious-XL-v0.1.safetensors" \
#     --vae="{SD_MODEL_PATH}/sdxl_fp16_fix_vae.safetensors" \
#     --train_data_dir="{INPUT_DATASET_PATH}/Nachoneko" \
#     --output_name="Nachoneko_2" \
#     --output_dir="{OUTPUT_PATH}/Nachoneko" \
#     --wandb_run_name="Nachoneko" \
#     --log_tracker_name="lora-Nachoneko" \
#     --prior_loss_weight=1 \
#     --resolution="1024,1024" \
#     --enable_bucket \
#     --min_bucket_reso=256 \
#     --max_bucket_reso=4096 \
#     --bucket_reso_steps=64 \
#     --save_model_as="safetensors" \
#     --save_precision="fp16" \
#     --save_every_n_epochs=1 \
#     --max_train_epochs=40 \
#     --train_batch_size=6 \
#     --gradient_checkpointing \
#     --network_train_unet_only \
#     --learning_rate=0.00012 \
#     --unet_lr=0.00012 \
#     --text_encoder_lr=0.00001 \
#     --lr_scheduler="constant_with_warmup" \
#     --lr_warmup_steps=100 \
#     --optimizer_type="Lion8bit" \
#     --network_module="lycoris.kohya" \
#     --network_dim=100000 \
#     --network_alpha=100000 \
#     --network_args \
#         conv_dim=100000 \
#         conv_alpha=100000 \
#         algo=lokr \
#         dropout=0 \
#         factor=8 \
#         train_norm=True \
#     --log_with="{LOG_MODULE}" \
#     --logging_dir="{OUTPUT_PATH}/logs" \
#     --caption_extension=".txt" \
#     --shuffle_caption \
#     --keep_tokens=0 \
#     --max_token_length=225 \
#     --seed=1337 \
#     --mixed_precision="fp16" \
#     --xformers \
#     --cache_latents \
#     --cache_latents_to_disk \
#     --persistent_data_loader_workers \
#     --vae_batch_size=4 \
#     --full_fp16


# 使用 lokr 算法训练 XL 画风 LoRA, 使用多卡进行训练
# 该参数也可以用于人物 LoRA 训练
# 
# 在训练多画风 LoRA 或者人物 LoRA 时, 通常会打上触发词
# 当使用了 --network_train_unet_only 后, Text Encoder 虽然不会训练, 但并不影响将触发词训练进 LoRA 模型中
# 并且不训练 Text Encoder 避免 Text Encoder 被炼烂(Text Encoder 比较容易被炼烂)
#
# 这个参数是在 Illustrious-XL-v0.1.safetensors 模型上测出来的, 大概在 32 Epoch 左右有比较好的效果
# 用 animagine-xl-3.1.safetensors 那套参数也有不错的效果, 只是学习率比这套低了点, 学得慢一点
# 学习率调度器从 cosine_with_restarts 换成 constant_with_warmup, 此时学习率靠优化器(Lion8bit)进行调度
# 拟合速度会更快
# constant_with_warmup 用在大规模的训练上比较好, 但用在小规模训练也有不错的效果
# 如果训练集的图比较少, 重复的图较多, 重复次数较高, 可能容易造成过拟合
# 
# 在 --network_args 设置了 preset，可以调整训练网络的大小
# 该值默认为 full，而使用 attn-mlp 可以得到更小的 LoRA 但几乎不影响 LoRA 效果
# 可用的预设可阅读文档: https://github.com/KohakuBlueleaf/LyCORIS/blob/main/docs/Preset.md
# 该预设也可以自行编写并指定, 编写例子可查看: https://github.com/KohakuBlueleaf/LyCORIS/blob/main/example_configs/preset_configs/example.toml
# 
# !python -m accelerate.commands.launch \
#     --num_cpu_threads_per_process=1 \
#     --multi_gpu \
#     --num_processes=2 \
#     "{SD_SCRIPTS_PATH}/sdxl_train_network.py" \
#     --pretrained_model_name_or_path="{SD_MODEL_PATH}/Illustrious-XL-v0.1.safetensors" \
#     --vae="{SD_MODEL_PATH}/sdxl_fp16_fix_vae.safetensors" \
#     --train_data_dir="{INPUT_DATASET_PATH}/Nachoneko" \
#     --output_name="Nachoneko_2" \
#     --output_dir="{OUTPUT_PATH}/Nachoneko" \
#     --wandb_run_name="Nachoneko" \
#     --log_tracker_name="lora-Nachoneko" \
#     --prior_loss_weight=1 \
#     --resolution="1024,1024" \
#     --enable_bucket \
#     --min_bucket_reso=256 \
#     --max_bucket_reso=4096 \
#     --bucket_reso_steps=64 \
#     --save_model_as="safetensors" \
#     --save_precision="fp16" \
#     --save_every_n_epochs=1 \
#     --max_train_epochs=40 \
#     --train_batch_size=6 \
#     --gradient_checkpointing \
#     --network_train_unet_only \
#     --learning_rate=0.00012 \
#     --unet_lr=0.00012 \
#     --text_encoder_lr=0.00001 \
#     --lr_scheduler="constant_with_warmup" \
#     --lr_warmup_steps=100 \
#     --optimizer_type="Lion8bit" \
#     --network_module="lycoris.kohya" \
#     --network_dim=100000 \
#     --network_alpha=100000 \
#     --network_args \
#         conv_dim=100000 \
#         conv_alpha=100000 \
#         algo=lokr \
#         dropout=0 \
#         factor=8 \
#         train_norm=True \
#         preset="attn-mlp" \
#     --log_with="{LOG_MODULE}" \
#     --logging_dir="{OUTPUT_PATH}/logs" \
#     --caption_extension=".txt" \
#     --shuffle_caption \
#     --keep_tokens=0 \
#     --max_token_length=225 \
#     --seed=1337 \
#     --mixed_precision="fp16" \
#     --xformers \
#     --cache_latents \
#     --cache_latents_to_disk \
#     --persistent_data_loader_workers \
#     --vae_batch_size=4 \
#     --full_fp16


# 使用 lokr 算法训练 XL 画风 LoRA, 使用多卡进行训练
# 该参数也可以用于人物 LoRA 训练
# 
# 在训练多画风 LoRA 或者人物 LoRA 时, 通常会打上触发词
# 当使用了 --network_train_unet_only 后, Text Encoder 虽然不会训练, 但并不影响将触发词训练进 LoRA 模型中
# 并且不训练 Text Encoder 避免 Text Encoder 被炼烂(Text Encoder 比较容易被炼烂)
#
# 这个参数是在 Illustrious-XL-v0.1.safetensors 模型上测出来的, 大概在 32 Epoch 左右有比较好的效果
# 用 animagine-xl-3.1.safetensors 那套参数也有不错的效果, 只是学习率比这套低了点, 学得慢一点
# 学习率调度器从 cosine_with_restarts 换成 constant_with_warmup, 此时学习率靠优化器(Lion8bit)进行调度
# 拟合速度会更快
# constant_with_warmup 用在大规模的训练上比较好, 但用在小规模训练也有不错的效果
# 如果训练集的图比较少, 重复的图较多, 重复次数较高, 可能容易造成过拟合
# 
# 在 --network_args 设置了 preset，可以调整训练网络的大小
# 该值默认为 full，而使用 attn-mlp 可以得到更小的 LoRA 但几乎不影响 LoRA 效果
# 可用的预设可阅读文档: https://github.com/KohakuBlueleaf/LyCORIS/blob/main/docs/Preset.md
# 该预设也可以自行编写并指定, 编写例子可查看: https://github.com/KohakuBlueleaf/LyCORIS/blob/main/example_configs/preset_configs/example.toml
# 
# 使用 --optimizer_args 设置 weight_decay 和 betas, 更高的 weight_decay 可以降低拟合程度, 减少过拟合
# 如果拟合程度不够高, 可以提高 --max_train_epochs 的值, 或者适当降低 weight_decay 的值, 可自行测试
# 较小的训练集适合使用较小的值, 如 0.05, 较大的训练集适合用 0.1
# 当 weight_decay 设置为 0.05 时, 大概在 38 Epoch 有比较好的效果
# 
# !python -m accelerate.commands.launch \
#     --num_cpu_threads_per_process=1 \
#     --multi_gpu \
#     --num_processes=2 \
#     "{SD_SCRIPTS_PATH}/sdxl_train_network.py" \
#     --pretrained_model_name_or_path="{SD_MODEL_PATH}/Illustrious-XL-v0.1.safetensors" \
#     --vae="{SD_MODEL_PATH}/sdxl_fp16_fix_vae.safetensors" \
#     --train_data_dir="{INPUT_DATASET_PATH}/Nachoneko" \
#     --output_name="Nachoneko_2" \
#     --output_dir="{OUTPUT_PATH}/Nachoneko" \
#     --wandb_run_name="Nachoneko" \
#     --log_tracker_name="lora-Nachoneko" \
#     --prior_loss_weight=1 \
#     --resolution="1024,1024" \
#     --enable_bucket \
#     --min_bucket_reso=256 \
#     --max_bucket_reso=4096 \
#     --bucket_reso_steps=64 \
#     --save_model_as="safetensors" \
#     --save_precision="fp16" \
#     --save_every_n_epochs=1 \
#     --max_train_epochs=40 \
#     --train_batch_size=6 \
#     --gradient_checkpointing \
#     --network_train_unet_only \
#     --learning_rate=0.00012 \
#     --unet_lr=0.00012 \
#     --text_encoder_lr=0.00001 \
#     --lr_scheduler="constant_with_warmup" \
#     --lr_warmup_steps=100 \
#     --optimizer_type="Lion8bit" \
#     --network_module="lycoris.kohya" \
#     --network_dim=100000 \
#     --network_alpha=100000 \
#     --network_args \
#         conv_dim=100000 \
#         conv_alpha=100000 \
#         algo=lokr \
#         dropout=0 \
#         factor=8 \
#         train_norm=True \
#         preset="attn-mlp" \
#     --optimizer_args \
#         weight_decay=0.1 \
#         betas="0.9,0.95" \
#     --log_with="{LOG_MODULE}" \
#     --logging_dir="{OUTPUT_PATH}/logs" \
#     --caption_extension=".txt" \
#     --shuffle_caption \
#     --keep_tokens=0 \
#     --max_token_length=225 \
#     --seed=1337 \
#     --mixed_precision="fp16" \
#     --xformers \
#     --cache_latents \
#     --cache_latents_to_disk \
#     --persistent_data_loader_workers \
#     --vae_batch_size=4 \
#     --full_fp16


# (自己在用的)
# 使用 lokr 算法训练 XL 画风 LoRA, 使用多卡进行训练
# 该参数也可以用于人物 LoRA 训练
# 
# 在训练多画风 LoRA 或者人物 LoRA 时, 通常会打上触发词
# 当使用了 --network_train_unet_only 后, Text Encoder 虽然不会训练, 但并不影响将触发词训练进 LoRA 模型中
# 并且不训练 Text Encoder 避免 Text Encoder 被炼烂(Text Encoder 比较容易被炼烂)
# 
# 学习率调度器从 cosine_with_restarts 换成 constant_with_warmup, 此时学习率靠优化器(Lion8bit)进行调度
# 拟合速度会更快
# constant_with_warmup 用在大规模的训练上比较好, 但用在小规模训练也有不错的效果
# 如果训练集的图比较少, 重复的图较多, 重复次数较高, 可能容易造成过拟合
# 
# 在 --network_args 设置了 preset, 可以调整训练网络的大小
# 该值默认为 full, 如果使用 attn-mlp 可以得到更小的 LoRA, 但对于难学的概念使用 full 效果会更好
# 
# 可用的预设可阅读文档: https://github.com/KohakuBlueleaf/LyCORIS/blob/main/docs/Preset.md
# 该预设也可以自行编写并指定, 编写例子可查看: https://github.com/KohakuBlueleaf/LyCORIS/blob/main/example_configs/preset_configs/example.toml
# 
# 使用 --optimizer_args 设置 weight_decay 和 betas, 更高的 weight_decay 可以降低拟合程度, 减少过拟合
# 如果拟合程度不够高, 可以提高 --max_train_epochs 的值, 或者适当降低 weight_decay 的值, 可自行测试
# 较小的训练集适合使用较小的值, 如 0.05, 较大的训练集适合用 0.1
# 大概 34 Epoch 会有比较好的效果吧, 不过不好说, 看训练集
# 自己测的时候大概在 26~40 Epoch 之间会出现好结果, 测试了很多炉基本都在这个区间里, 但也不排除意外情况 (训练参数这东西好麻烦啊, 苦い)
# 
# !python -m accelerate.commands.launch \
#     --num_cpu_threads_per_process=1 \
#     --multi_gpu \
#     --num_processes=2 \
#     "{SD_SCRIPTS_PATH}/sdxl_train_network.py" \
#     --pretrained_model_name_or_path="{SD_MODEL_PATH}/Illustrious-XL-v0.1.safetensors" \
#     --vae="{SD_MODEL_PATH}/sdxl_fp16_fix_vae.safetensors" \
#     --train_data_dir="{INPUT_DATASET_PATH}/Nachoneko" \
#     --output_name="Nachoneko_2" \
#     --output_dir="{OUTPUT_PATH}/Nachoneko" \
#     --wandb_run_name="Nachoneko" \
#     --log_tracker_name="lora-Nachoneko" \
#     --prior_loss_weight=1 \
#     --resolution="1024,1024" \
#     --enable_bucket \
#     --min_bucket_reso=256 \
#     --max_bucket_reso=4096 \
#     --bucket_reso_steps=64 \
#     --save_model_as="safetensors" \
#     --save_precision="fp16" \
#     --save_every_n_epochs=1 \
#     --max_train_epochs=40 \
#     --train_batch_size=6 \
#     --gradient_checkpointing \
#     --network_train_unet_only \
#     --learning_rate=0.0001 \
#     --unet_lr=0.0001 \
#     --text_encoder_lr=0.00001 \
#     --lr_scheduler="constant_with_warmup" \
#     --lr_warmup_steps=100 \
#     --optimizer_type="Lion8bit" \
#     --network_module="lycoris.kohya" \
#     --network_dim=100000 \
#     --network_alpha=100000 \
#     --network_args \
#         conv_dim=100000 \
#         conv_alpha=100000 \
#         algo=lokr \
#         dropout=0 \
#         factor=8 \
#         train_norm=True \
#         preset="full" \
#     --optimizer_args \
#         weight_decay=0.05 \
#         betas="0.9,0.95" \
#     --log_with="{LOG_MODULE}" \
#     --logging_dir="{OUTPUT_PATH}/logs" \
#     --caption_extension=".txt" \
#     --shuffle_caption \
#     --keep_tokens=0 \
#     --max_token_length=225 \
#     --seed=1337 \
#     --mixed_precision="fp16" \
#     --xformers \
#     --cache_latents \
#     --cache_latents_to_disk \
#     --persistent_data_loader_workers \
#     --vae_batch_size=4 \
#     --full_fp16


# (自己在用的)
# 使用 lokr 算法训练 XL 画风 LoRA, 使用多卡进行训练
# 该参数也可以用于人物 LoRA 训练
# 
# 在训练多画风 LoRA 或者人物 LoRA 时, 通常会打上触发词
# 当使用了 --network_train_unet_only 后, Text Encoder 虽然不会训练, 但并不影响将触发词训练进 LoRA 模型中
# 并且不训练 Text Encoder 避免 Text Encoder 被炼烂(Text Encoder 比较容易被炼烂)
# 
# 学习率调度器从 cosine_with_restarts 换成 constant_with_warmup, 此时学习率靠优化器(Lion8bit)进行调度
# 拟合速度会更快
# constant_with_warmup 用在大规模的训练上比较好, 但用在小规模训练也有不错的效果
# 如果训练集的图比较少, 重复的图较多, 重复次数较高, 可能容易造成过拟合
# 
# 在 --network_args 设置了 preset, 可以调整训练网络的大小
# 该值默认为 full, 如果使用 attn-mlp 可以得到更小的 LoRA, 但对于难学的概念使用 full 效果会更好 (最好还是 full 吧, 其他的预设效果不是很好)
# 
# 可用的预设可阅读文档: https://github.com/KohakuBlueleaf/LyCORIS/blob/main/docs/Preset.md
# 该预设也可以自行编写并指定, 编写例子可查看: https://github.com/KohakuBlueleaf/LyCORIS/blob/main/example_configs/preset_configs/example.toml
# 
# 使用 --optimizer_args 设置 weight_decay 和 betas, 更高的 weight_decay 可以降低拟合程度, 减少过拟合
# 如果拟合程度不够高, 可以提高 --max_train_epochs 的值, 或者适当降低 weight_decay 的值, 可自行测试
# 较小的训练集适合使用较小的值, 如 0.05, 较大的训练集适合用 0.1
# 大概 34 Epoch 会有比较好的效果吧, 不过不好说, 看训练集
# 自己测的时候大概在 26~40 Epoch 之间会出现好结果, 测试了很多炉基本都在这个区间里, 但也不排除意外情况 (训练参数这东西好麻烦啊, 苦い)
# 
# 测试的时候发现 --debiased_estimation_loss 对于训练效果的有些改善
# 这里有个对比: https://licyk.netlify.app/2025/02/10/debiased_estimation_loss_in_stable_diffusion_model_training
# 启用后能提高拟合速度和颜色表现吧, 画风的学习能学得更好
# 但, 肢体崩坏率可能会有点提高, 不过有另一套参数去优化了一下这个问题, 貌似会好一点
# 可能画风会弱化, 所以不是很确定哪个比较好用, 只能自己试了
# debiased estimation loss 有个相关的论文可以看看: https://arxiv.org/abs/2310.08442
# 
# !python -m accelerate.commands.launch \
#     --num_cpu_threads_per_process=1 \
#     --multi_gpu \
#     --num_processes=2 \
#     "{SD_SCRIPTS_PATH}/sdxl_train_network.py" \
#     --pretrained_model_name_or_path="{SD_MODEL_PATH}/Illustrious-XL-v0.1.safetensors" \
#     --vae="{SD_MODEL_PATH}/sdxl_fp16_fix_vae.safetensors" \
#     --train_data_dir="{INPUT_DATASET_PATH}/Nachoneko" \
#     --output_name="Nachoneko_2" \
#     --output_dir="{OUTPUT_PATH}/Nachoneko" \
#     --wandb_run_name="Nachoneko" \
#     --log_tracker_name="lora-Nachoneko" \
#     --prior_loss_weight=1 \
#     --resolution="1024,1024" \
#     --enable_bucket \
#     --min_bucket_reso=256 \
#     --max_bucket_reso=4096 \
#     --bucket_reso_steps=64 \
#     --save_model_as="safetensors" \
#     --save_precision="fp16" \
#     --save_every_n_epochs=1 \
#     --max_train_epochs=40 \
#     --train_batch_size=6 \
#     --gradient_checkpointing \
#     --network_train_unet_only \
#     --learning_rate=0.0001 \
#     --unet_lr=0.0001 \
#     --text_encoder_lr=0.00001 \
#     --lr_scheduler="constant_with_warmup" \
#     --lr_warmup_steps=100 \
#     --optimizer_type="Lion8bit" \
#     --network_module="lycoris.kohya" \
#     --network_dim=100000 \
#     --network_alpha=100000 \
#     --network_args \
#         conv_dim=100000 \
#         conv_alpha=100000 \
#         algo=lokr \
#         dropout=0 \
#         factor=8 \
#         train_norm=True \
#         preset="full" \
#     --optimizer_args \
#         weight_decay=0.05 \
#         betas="0.9,0.95" \
#     --log_with="{LOG_MODULE}" \
#     --logging_dir="{OUTPUT_PATH}/logs" \
#     --caption_extension=".txt" \
#     --shuffle_caption \
#     --keep_tokens=0 \
#     --max_token_length=225 \
#     --seed=1337 \
#     --mixed_precision="fp16" \
#     --xformers \
#     --cache_latents \
#     --cache_latents_to_disk \
#     --persistent_data_loader_workers \
#     --debiased_estimation_loss \
#     --vae_batch_size=4 \
#     --full_fp16


# (自己在用的)
# 使用 lokr 算法训练 XL 画风 LoRA, 使用多卡进行训练
# 该参数也可以用于人物 LoRA 训练
# 
# 在训练多画风 LoRA 或者人物 LoRA 时, 通常会打上触发词
# 当使用了 --network_train_unet_only 后, Text Encoder 虽然不会训练, 但并不影响将触发词训练进 LoRA 模型中
# 并且不训练 Text Encoder 避免 Text Encoder 被炼烂(Text Encoder 比较容易被炼烂)
# 
# 学习率调度器从 cosine_with_restarts 换成 constant_with_warmup, 此时学习率靠优化器(Lion8bit)进行调度
# 拟合速度会更快
# constant_with_warmup 用在大规模的训练上比较好, 但用在小规模训练也有不错的效果
# 如果训练集的图比较少, 重复的图较多, 重复次数较高, 可能容易造成过拟合
# 
# 在 --network_args 设置了 preset, 可以调整训练网络的大小
# 该值默认为 full, 如果使用 attn-mlp 可以得到更小的 LoRA, 但对于难学的概念使用 full 效果会更好 (最好还是 full 吧, 其他的预设效果不是很好)
# 
# 可用的预设可阅读文档: https://github.com/KohakuBlueleaf/LyCORIS/blob/main/docs/Preset.md
# 该预设也可以自行编写并指定, 编写例子可查看: https://github.com/KohakuBlueleaf/LyCORIS/blob/main/example_configs/preset_configs/example.toml
# 
# 使用 --optimizer_args 设置 weight_decay 和 betas, 更高的 weight_decay 可以降低拟合程度, 减少过拟合
# 如果拟合程度不够高, 可以提高 --max_train_epochs 的值, 或者适当降低 weight_decay 的值, 可自行测试
# 较小的训练集适合使用较小的值, 如 0.05, 较大的训练集适合用 0.1
# 大概 34 Epoch 会有比较好的效果吧, 不过不好说, 看训练集
# 自己测的时候大概在 26~40 Epoch 之间会出现好结果, 测试了很多炉基本都在这个区间里, 但也不排除意外情况 (训练参数这东西好麻烦啊, 苦い)
# 
# 测试的时候发现 --debiased_estimation_loss 对于训练效果的有些改善
# 这里有个对比: https://licyk.netlify.app/2025/02/10/debiased_estimation_loss_in_stable_diffusion_model_training
# 启用后能提高拟合速度和颜色表现吧, 画风的学习能学得更好
# 但, 肢体崩坏率可能会有点提高, 不过有另一套参数去优化了一下这个问题, 貌似会好一点
# 可能画风会弱化, 所以不是很确定哪个比较好用, 只能自己试了
# debiased estimation loss 有个相关的论文可以看看: https://arxiv.org/abs/2310.08442
# 
# 加上 v 预测参数进行训练, 提高模型对暗处和亮处的表现效果, 并且能让模型能够直出纯黑色背景, 画面也更干净
# 相关的论文可以看看: https://arxiv.org/abs/2305.08891
# 
# !python -m accelerate.commands.launch \
#     --num_cpu_threads_per_process=1 \
#     --multi_gpu \
#     --num_processes=2 \
#     "{SD_SCRIPTS_PATH}/sdxl_train_network.py" \
#     --pretrained_model_name_or_path="{SD_MODEL_PATH}/noobaiXLNAIXL_vPred10Version.safetensors" \
#     --vae="{SD_MODEL_PATH}/sdxl_fp16_fix_vae.safetensors" \
#     --train_data_dir="{INPUT_DATASET_PATH}/Nachoneko" \
#     --output_name="Nachoneko_2" \
#     --output_dir="{OUTPUT_PATH}/Nachoneko" \
#     --wandb_run_name="Nachoneko" \
#     --log_tracker_name="lora-Nachoneko" \
#     --prior_loss_weight=1 \
#     --resolution="1024,1024" \
#     --enable_bucket \
#     --min_bucket_reso=256 \
#     --max_bucket_reso=4096 \
#     --bucket_reso_steps=64 \
#     --save_model_as="safetensors" \
#     --save_precision="fp16" \
#     --save_every_n_epochs=1 \
#     --max_train_epochs=40 \
#     --train_batch_size=6 \
#     --gradient_checkpointing \
#     --network_train_unet_only \
#     --learning_rate=0.0001 \
#     --unet_lr=0.0001 \
#     --text_encoder_lr=0.00001 \
#     --lr_scheduler="constant_with_warmup" \
#     --lr_warmup_steps=100 \
#     --optimizer_type="Lion8bit" \
#     --network_module="lycoris.kohya" \
#     --network_dim=100000 \
#     --network_alpha=100000 \
#     --network_args \
#         conv_dim=100000 \
#         conv_alpha=100000 \
#         algo=lokr \
#         dropout=0 \
#         factor=8 \
#         train_norm=True \
#         preset="full" \
#     --optimizer_args \
#         weight_decay=0.05 \
#         betas="0.9,0.95" \
#     --log_with="{LOG_MODULE}" \
#     --logging_dir="{OUTPUT_PATH}/logs" \
#     --caption_extension=".txt" \
#     --shuffle_caption \
#     --keep_tokens=0 \
#     --max_token_length=225 \
#     --seed=1337 \
#     --mixed_precision="fp16" \
#     --xformers \
#     --cache_latents \
#     --cache_latents_to_disk \
#     --persistent_data_loader_workers \
#     --debiased_estimation_loss \
#     --vae_batch_size=4 \
#     --zero_terminal_snr \
#     --v_parameterization \
#     --scale_v_pred_loss_like_noise_pred \
#     --full_fp16


# 使用 lokr 算法训练 XL 画风 LoRA, 使用多卡进行训练
# 该参数也可以用于人物 LoRA 训练
# 
# 在训练多画风 LoRA 或者人物 LoRA 时, 通常会打上触发词
# 当使用了 --network_train_unet_only 后, Text Encoder 虽然不会训练, 但并不影响将触发词训练进 LoRA 模型中
# 并且不训练 Text Encoder 避免 Text Encoder 被炼烂(Text Encoder 比较容易被炼烂)
# 
# 在 --network_args 设置了 preset, 可以调整训练网络的大小
# 该值默认为 full, 如果使用 attn-mlp 可以得到更小的 LoRA, 但对于难学的概念使用 full 效果会更好 (最好还是 full 吧, 其他的预设效果不是很好)
# 
# 可用的预设可阅读文档: https://github.com/KohakuBlueleaf/LyCORIS/blob/main/docs/Preset.md
# 该预设也可以自行编写并指定, 编写例子可查看: https://github.com/KohakuBlueleaf/LyCORIS/blob/main/example_configs/preset_configs/example.toml
# 
# 使用 --optimizer_args 设置 weight_decay 和 betas, 更高的 weight_decay 可以降低拟合程度, 减少过拟合
# 如果拟合程度不够高, 可以提高 --max_train_epochs 的值, 或者适当降低 weight_decay 的值, 可自行测试
# 较小的训练集适合使用较小的值, 如 0.05, 较大的训练集适合用 0.1
# 大概 34 Epoch 会有比较好的效果吧, 不过不好说, 看训练集
# 自己测的时候大概在 26~40 Epoch 之间会出现好结果, 测试了很多炉基本都在这个区间里, 但也不排除意外情况 (训练参数这东西好麻烦啊, 苦い)
# 
# 测试的时候发现 --debiased_estimation_loss 对于训练效果的有些改善
# 这里有个对比: https://licyk.netlify.app/2025/02/10/debiased_estimation_loss_in_stable_diffusion_model_training
# 启用后能提高拟合速度和颜色表现吧, 画风的学习能学得更好
# 把学习率调度器 constant_with_warmup 换成了cosine, 稍微缓解了一下拟合速度过快导致肢体崩坏率增大的问题
# 如果学的效果不够好, 拟合度不够高, 可以适当增加 --max_train_epochs 的值或者提高训练集的重复次数
# debiased estimation loss 有个相关的论文可以看看: https://arxiv.org/abs/2310.08442
# 
# !python -m accelerate.commands.launch \
#     --num_cpu_threads_per_process=1 \
#     --multi_gpu \
#     --num_processes=2 \
#     "{SD_SCRIPTS_PATH}/sdxl_train_network.py" \
#     --pretrained_model_name_or_path="{SD_MODEL_PATH}/Illustrious-XL-v0.1.safetensors" \
#     --vae="{SD_MODEL_PATH}/sdxl_fp16_fix_vae.safetensors" \
#     --train_data_dir="{INPUT_DATASET_PATH}/Nachoneko" \
#     --output_name="Nachoneko_2" \
#     --output_dir="{OUTPUT_PATH}/Nachoneko" \
#     --wandb_run_name="Nachoneko" \
#     --log_tracker_name="lora-Nachoneko" \
#     --prior_loss_weight=1 \
#     --resolution="1024,1024" \
#     --enable_bucket \
#     --min_bucket_reso=256 \
#     --max_bucket_reso=4096 \
#     --bucket_reso_steps=64 \
#     --save_model_as="safetensors" \
#     --save_precision="fp16" \
#     --save_every_n_epochs=1 \
#     --max_train_epochs=40 \
#     --train_batch_size=6 \
#     --gradient_checkpointing \
#     --network_train_unet_only \
#     --learning_rate=0.0001 \
#     --unet_lr=0.0001 \
#     --text_encoder_lr=0.00001 \
#     --lr_scheduler="cosine" \
#     --lr_warmup_steps=0 \
#     --optimizer_type="Lion8bit" \
#     --network_module="lycoris.kohya" \
#     --network_dim=100000 \
#     --network_alpha=100000 \
#     --network_args \
#         conv_dim=100000 \
#         conv_alpha=100000 \
#         algo=lokr \
#         dropout=0 \
#         factor=8 \
#         train_norm=True \
#         preset="full" \
#     --optimizer_args \
#         weight_decay=0.05 \
#         betas="0.9,0.95" \
#     --log_with="{LOG_MODULE}" \
#     --logging_dir="{OUTPUT_PATH}/logs" \
#     --caption_extension=".txt" \
#     --shuffle_caption \
#     --keep_tokens=0 \
#     --max_token_length=225 \
#     --seed=1337 \
#     --mixed_precision="fp16" \
#     --xformers \
#     --cache_latents \
#     --cache_latents_to_disk \
#     --persistent_data_loader_workers \
#     --debiased_estimation_loss \
#     --vae_batch_size=4 \
#     --full_fp16


# 使用 lokr 算法训练 XL 人物 LoRA, 使用多卡进行训练
# 适合极少图或者单图训练集进行人物 LoRA 训练
# 训练集使用打标器进行打标后, 要保留的人物的哪些特征, 就把对应的 Tag 删去, 触发词可加可不加
# 
# 该参数使用 scale_weight_norms 降低过拟合程度, 进行训练时, 可在控制台输出看到 Average key norm 这个值
# 通常测试 LoRA 时就测试 Average key norm 值在 0.5 ~ 0.9 之间的保存的 LoRA 模型
# max_train_epochs 设置为 200, save_every_n_epochs 设置为 1 以为了更好的挑选最好的结果
# 
# 可使用该方法训练一个人物 LoRA 模型用于生成人物的图片, 并将这些图片重新制作成训练集
# 再使用不带 scale_weight_norms 的训练参数进行训练, 通过这种方式, 可以在图片极少的情况下得到比较好的 LoRA 模型
# 
# !python -m accelerate.commands.launch \
#     --num_cpu_threads_per_process=1 \
#     --multi_gpu \
#     --num_processes=2 \
#     "{SD_SCRIPTS_PATH}/sdxl_train_network.py" \
#     --pretrained_model_name_or_path="{SD_MODEL_PATH}/Illustrious-XL-v0.1.safetensors" \
#     --vae="{SD_MODEL_PATH}/sdxl_fp16_fix_vae.safetensors" \
#     --train_data_dir="{INPUT_DATASET_PATH}/Nachoneko" \
#     --output_name="Nachoneko_2" \
#     --output_dir="{OUTPUT_PATH}/Nachoneko" \
#     --wandb_run_name="Nachoneko" \
#     --log_tracker_name="lora-Nachoneko" \
#     --prior_loss_weight=1 \
#     --resolution="1024,1024" \
#     --enable_bucket \
#     --min_bucket_reso=256 \
#     --max_bucket_reso=4096 \
#     --bucket_reso_steps=64 \
#     --save_model_as="safetensors" \
#     --save_precision="fp16" \
#     --save_every_n_epochs=1 \
#     --max_train_epochs=200 \
#     --train_batch_size=6 \
#     --gradient_checkpointing \
#     --network_train_unet_only \
#     --learning_rate=0.00012 \
#     --unet_lr=0.00012 \
#     --text_encoder_lr=0.00001 \
#     --lr_scheduler="constant_with_warmup" \
#     --lr_warmup_steps=1 \
#     --optimizer_type="Lion8bit" \
#     --network_module="lycoris.kohya" \
#     --scale_weight_norms=1 \
#     --network_dim=100000 \
#     --network_alpha=100000 \
#     --network_args \
#         conv_dim=100000 \
#         conv_alpha=100000 \
#         algo=lokr \
#         dropout=0 \
#         factor=8 \
#         train_norm=True \
#     --log_with="{LOG_MODULE}" \
#     --logging_dir="{OUTPUT_PATH}/logs" \
#     --caption_extension=".txt" \
#     --shuffle_caption \
#     --keep_tokens=0 \
#     --max_token_length=225 \
#     --seed=1337 \
#     --mixed_precision="fp16" \
#     --xformers \
#     --cache_latents \
#     --cache_latents_to_disk \
#     --persistent_data_loader_workers \
#     --vae_batch_size=4 \
#     --full_fp16


# 使用 lokr 算法训练 XL 画风 LoRA, 使用 rtx 4060 8g laptop 进行训练, 通过 fp8 降低显存占用
# 该参数也可以用于人物 LoRA 训练
# 
# 在训练多画风 LoRA 或者人物 LoRA 时, 通常会打上触发词
# 当使用了 --network_train_unet_only 后, Text Encoder 虽然不会训练, 但并不影响将触发词训练进 LoRA 模型中
# 并且不训练 Text Encoder 避免 Text Encoder 被炼烂(Text Encoder 比较容易被炼烂)
#
# 这个参数是在 Illustrious-XL-v0.1.safetensors 模型上测出来的, 大概在 32 Epoch 左右有比较好的效果
# 用 animagine-xl-3.1.safetensors 那套参数也有不错的效果, 只是学习率比这套低了点, 学得慢一点
# 学习率调度器从 cosine_with_restarts 换成 constant_with_warmup, 此时学习率靠优化器(Lion8bit)进行调度
# 拟合速度会更快
# constant_with_warmup 用在大规模的训练上比较好, 但用在小规模训练也有不错的效果
# 如果训练集的图比较少, 重复的图较多, 重复次数较高, 可能容易造成过拟合
# 
# !python "{SD_SCRIPTS_PATH}/sdxl_train_network.py" \
#     --pretrained_model_name_or_path="{SD_MODEL_PATH}/Illustrious-XL-v0.1.safetensors" \
#     --vae="{SD_MODEL_PATH}/sdxl_fp16_fix_vae.safetensors" \
#     --train_data_dir="{INPUT_DATASET_PATH}/Nachoneko" \
#     --output_name="Nachoneko_2" \
#     --output_dir="{OUTPUT_PATH}/Nachoneko" \
#     --wandb_run_name="Nachoneko" \
#     --log_tracker_name="lora-Nachoneko" \
#     --prior_loss_weight=1 \
#     --resolution="1024,1024" \
#     --enable_bucket \
#     --min_bucket_reso=256 \
#     --max_bucket_reso=4096 \
#     --bucket_reso_steps=64 \
#     --save_model_as="safetensors" \
#     --save_precision="fp16" \
#     --save_every_n_epochs=1 \
#     --max_train_epochs=40 \
#     --train_batch_size=3 \
#     --gradient_checkpointing \
#     --network_train_unet_only \
#     --learning_rate=0.0002 \
#     --unet_lr=0.0002 \
#     --text_encoder_lr=0.00001 \
#     --lr_scheduler="constant_with_warmup" \
#     --lr_warmup_steps=100 \
#     --optimizer_type="AdamW8bit" \
#     --network_module="lycoris.kohya" \
#     --network_dim=100000 \
#     --network_alpha=100000 \
#     --network_args \
#         conv_dim=100000 \
#         conv_alpha=100000 \
#         algo=lokr \
#         dropout=0 \
#         factor=8 \
#         train_norm=True \
#     --log_with="{LOG_MODULE}" \
#     --logging_dir="{OUTPUT_PATH}/logs" \
#     --caption_extension=".txt" \
#     --shuffle_caption \
#     --keep_tokens=0 \
#     --max_token_length=225 \
#     --seed=1337 \
#     --mixed_precision="fp16" \
#     --xformers \
#     --cache_latents \
#     --cache_latents_to_disk \
#     --persistent_data_loader_workers \
#     --vae_batch_size=4 \
#     --fp8_base


# 使用 lokr 算法训练 XL 画风 LoRA, 使用多卡进行训练
# 该参数也可以用于人物 LoRA 训练
# 
# 在训练多画风 LoRA 或者人物 LoRA 时, 通常会打上触发词
# 当使用了 --network_train_unet_only 后, Text Encoder 虽然不会训练, 但并不影响将触发词训练进 LoRA 模型中
# 并且不训练 Text Encoder 避免 Text Encoder 被炼烂(Text Encoder 比较容易被炼烂)
#
# 这个参数是在 Illustrious-XL-v0.1.safetensors 模型上测出来的, 大概在 32 Epoch 左右有比较好的效果
# 用 animagine-xl-3.1.safetensors 那套参数也有不错的效果, 只是学习率比这套低了点, 学得慢一点
# 学习率调度器从 cosine_with_restarts 换成 constant_with_warmup, 此时学习率靠优化器(Lion8bit)进行调度
# 拟合速度会更快
# constant_with_warmup 用在大规模的训练上比较好, 但用在小规模训练也有不错的效果
# 如果训练集的图比较少, 重复的图较多, 重复次数较高, 可能容易造成过拟合
# 
# 参数加上了 noise_offset, 可以提高暗处和亮处的表现, 一般使用设置成 0.05 ~ 0.1
# 但 noise_offset 可能会导致画面泛白, 光影效果变差
# 
# !python -m accelerate.commands.launch \
#     --num_cpu_threads_per_process=1 \
#     --multi_gpu \
#     --num_processes=2 \
#     "{SD_SCRIPTS_PATH}/sdxl_train_network.py" \
#     --pretrained_model_name_or_path="{SD_MODEL_PATH}/Illustrious-XL-v0.1.safetensors" \
#     --vae="{SD_MODEL_PATH}/sdxl_fp16_fix_vae.safetensors" \
#     --train_data_dir="{INPUT_DATASET_PATH}/Nachoneko" \
#     --output_name="Nachoneko_2" \
#     --output_dir="{OUTPUT_PATH}/Nachoneko" \
#     --wandb_run_name="Nachoneko" \
#     --log_tracker_name="lora-Nachoneko" \
#     --prior_loss_weight=1 \
#     --resolution="1024,1024" \
#     --enable_bucket \
#     --min_bucket_reso=256 \
#     --max_bucket_reso=4096 \
#     --bucket_reso_steps=64 \
#     --save_model_as="safetensors" \
#     --save_precision="fp16" \
#     --save_every_n_epochs=1 \
#     --max_train_epochs=40 \
#     --train_batch_size=6 \
#     --gradient_checkpointing \
#     --network_train_unet_only \
#     --learning_rate=0.00012 \
#     --unet_lr=0.00012 \
#     --text_encoder_lr=0.00001 \
#     --lr_scheduler="constant_with_warmup" \
#     --lr_warmup_steps=100 \
#     --optimizer_type="Lion8bit" \
#     --network_module="lycoris.kohya" \
#     --network_dim=100000 \
#     --network_alpha=100000 \
#     --network_args \
#         conv_dim=100000 \
#         conv_alpha=100000 \
#         algo=lokr \
#         dropout=0 \
#         factor=8 \
#         train_norm=True \
#     --log_with="{LOG_MODULE}" \
#     --logging_dir="{OUTPUT_PATH}/logs" \
#     --caption_extension=".txt" \
#     --shuffle_caption \
#     --keep_tokens=0 \
#     --max_token_length=225 \
#     --noise_offset=0.1 \
#     --seed=1337 \
#     --mixed_precision="fp16" \
#     --xformers \
#     --cache_latents \
#     --cache_latents_to_disk \
#     --persistent_data_loader_workers \
#     --vae_batch_size=4 \
#     --full_fp16


# 使用 lokr 算法训练 XL 人物 LoRA, 使用多卡进行训练
# 参数中使用了 --scale_weight_norms, 用于提高泛化性, 但可能会造成拟合度降低
# 如果当训练人物 LoRA 的图片较多时, 可考虑删去该参数
# 当训练人物 LoRA 的图片较少, 为了避免过拟合, 就可以考虑使用 --scale_weight_norms 降低过拟合概率
#
# !python -m accelerate.commands.launch \
#     --num_cpu_threads_per_process=1 \
#     --multi_gpu \
#     --num_processes=2 \
#     "{SD_SCRIPTS_PATH}/sdxl_train_network.py" \
#     --pretrained_model_name_or_path="{SD_MODEL_PATH}/animagine-xl-3.1.safetensors" \
#     --vae="{SD_MODEL_PATH}/sdxl_fp16_fix_vae.safetensors" \
#     --train_data_dir="{INPUT_DATASET_PATH}/robin" \
#     --output_name="robin_1" \
#     --output_dir="{OUTPUT_PATH}/robin_1" \
#     --wandb_run_name="Nachoneko" \
#     --log_tracker_name="lora-Nachoneko" \
#     --prior_loss_weight=1 \
#     --resolution="1024,1024" \
#     --enable_bucket \
#     --min_bucket_reso=256 \
#     --max_bucket_reso=4096 \
#     --bucket_reso_steps=64 \
#     --save_model_as="safetensors" \
#     --save_precision="fp16" \
#     --save_every_n_epochs=1 \
#     --max_train_epochs=50 \
#     --train_batch_size=6 \
#     --gradient_checkpointing \
#     --learning_rate=0.0001 \
#     --unet_lr=0.0001 \
#     --text_encoder_lr=0.00001 \
#     --lr_scheduler="cosine_with_restarts" \
#     --lr_warmup_steps=0 \
#     --lr_scheduler_num_cycles=1 \
#     --optimizer_type="Lion8bit" \
#     --network_module="lycoris.kohya" \
#     --scale_weight_norms=1 \
#     --network_dim=100000 \
#     --network_alpha=100000 \
#     --network_args \
#         conv_dim=100000 \
#         conv_alpha=100000 \
#         algo=lokr \
#         dropout=0 \
#         factor=8 \
#         train_norm=True \
#     --log_with="{LOG_MODULE}" \
#     --logging_dir="{OUTPUT_PATH}/logs" \
#     --caption_extension=".txt" \
#     --shuffle_caption \
#     --keep_tokens=0 \
#     --max_token_length=225 \
#     --seed=1337 \
#     --mixed_precision="fp16" \
#     --xformers \
#     --cache_latents \
#     --cache_latents_to_disk \
#     --persistent_data_loader_workers \
#     --vae_batch_size=4 \
#     --full_fp16


# 使用 lokr 算法训练 XL 人物 LoRA, 使用多卡进行训练
# !python -m accelerate.commands.launch \
#     --num_cpu_threads_per_process=1 \
#     --multi_gpu \
#     --num_processes=2 \
#     "{SD_SCRIPTS_PATH}/sdxl_train_network.py" \
#     --pretrained_model_name_or_path="{SD_MODEL_PATH}/animagine-xl-3.1.safetensors" \
#     --vae="{SD_MODEL_PATH}/sdxl_fp16_fix_vae.safetensors" \
#     --train_data_dir="{INPUT_DATASET_PATH}/murasame_(senren)_3" \
#     --output_name="murasame_(senren)_10" \
#     --output_dir="{OUTPUT_PATH}/murasame_(senren)_10" \
#     --wandb_run_name="Nachoneko" \
#     --log_tracker_name="lora-Nachoneko" \
#     --prior_loss_weight=1 \
#     --resolution="1024,1024" \
#     --enable_bucket \
#     --min_bucket_reso=256 \
#     --max_bucket_reso=4096 \
#     --bucket_reso_steps=64 \
#     --save_model_as="safetensors" \
#     --save_precision="fp16" \
#     --save_every_n_epochs=1 \
#     --max_train_epochs=50 \
#     --train_batch_size=6 \
#     --gradient_checkpointing \
#     --learning_rate=0.0001 \
#     --unet_lr=0.0001 \
#     --text_encoder_lr=0.00004 \
#     --lr_scheduler="cosine_with_restarts" \
#     --lr_warmup_steps=0 \
#     --lr_scheduler_num_cycles=1 \
#     --optimizer_type="Lion8bit" \
#     --network_module="lycoris.kohya" \
#     --scale_weight_norms=1 \
#     --network_dim=100000 \
#     --network_alpha=100000 \
#     --network_args \
#         conv_dim=100000 \
#         conv_alpha=100000 \
#         algo=lokr \
#         dropout=0 \
#         factor=8 \
#     --log_with="{LOG_MODULE}" \
#     --logging_dir="{OUTPUT_PATH}/logs" \
#     --caption_extension=".txt" \
#     --shuffle_caption \
#     --keep_tokens=0 \
#     --max_token_length=225 \
#     --seed=1337 \
#     --mixed_precision="fp16" \
#     --xformers \
#     --cache_latents \
#     --cache_latents_to_disk \
#     --persistent_data_loader_workers \
#     --vae_batch_size=4 \
#     --full_fp16


# 使用 lokr 算法训练 XL 画风 LoRA, 使用单卡进行训练 (Kaggle 的单 Tesla P100 性能不如双 Tesla T4, 建议使用双卡训练)
# !python "{SD_SCRIPTS_PATH}/sdxl_train_network.py" \
#     --pretrained_model_name_or_path="{SD_MODEL_PATH}/animagine-xl-3.1.safetensors" \
#     --vae="{SD_MODEL_PATH}/sdxl_fp16_fix_vae.safetensors" \
#     --train_data_dir="{INPUT_DATASET_PATH}/rafa" \
#     --output_name="rafa_1" \
#     --output_dir="{OUTPUT_PATH}/rafa" \
#     --wandb_run_name="Nachoneko" \
#     --log_tracker_name="lora-Nachoneko" \
#     --prior_loss_weight=1 \
#     --resolution="1024,1024" \
#     --enable_bucket \
#     --min_bucket_reso=256 \
#     --max_bucket_reso=4096 \
#     --bucket_reso_steps=64 \
#     --save_model_as="safetensors" \
#     --save_precision="fp16" \
#     --save_every_n_epochs=1 \
#     --max_train_epochs=50 \
#     --train_batch_size=6 \
#     --gradient_checkpointing \
#     --network_train_unet_only \
#     --learning_rate=0.00007 \
#     --unet_lr=0.00007 \
#     --text_encoder_lr=0.00001 \
#     --lr_scheduler="cosine_with_restarts" \
#     --lr_warmup_steps=0 \
#     --lr_scheduler_num_cycles=1 \
#     --optimizer_type="Lion8bit" \
#     --network_module="lycoris.kohya" \
#     --network_dim=100000 \
#     --network_alpha=100000 \
#     --network_args \
#         conv_dim=100000 \
#         conv_alpha=100000 \
#         algo=lokr \
#         dropout=0 \
#         factor=8 \
#         train_norm=True \
#     --log_with="{LOG_MODULE}" \
#     --logging_dir="{OUTPUT_PATH}/logs" \
#     --caption_extension=".txt" \
#     --shuffle_caption \
#     --keep_tokens=0 \
#     --max_token_length=225 \
#     --seed=1337 \
#     --mixed_precision="fp16" \
#     --xformers \
#     --cache_latents \
#     --cache_latents_to_disk \
#     --persistent_data_loader_workers \
#     --vae_batch_size=4 \
#     --full_fp16


# 使用 lokr 算法训练 SD1.5 画风 LoRA, 使用双卡进行训练
# 使用 NovelAI 1 模型进行训练
# 
# !python -m accelerate.commands.launch \
#     --num_cpu_threads_per_process=1 \
#     --multi_gpu \
#     --num_processes=2 \
#     "{SD_SCRIPTS_PATH}/train_network.py" \
#     --pretrained_model_name_or_path="{SD_MODEL_PATH}/animefull-final-pruned.safetensors" \
#     --vae="{SD_MODEL_PATH}/vae-ft-mse-840000-ema-pruned.safetensors" \
#     --train_data_dir="{INPUT_DATASET_PATH}/sunfish" \
#     --output_name="nai1-sunfish_5" \
#     --output_dir="{OUTPUT_PATH}/nai1-sunfish_5" \
#     --wandb_run_name="Nachoneko" \
#     --log_tracker_name="lora-Nachoneko" \
#     --prior_loss_weight=1 \
#     --resolution="768,768" \
#     --enable_bucket \
#     --min_bucket_reso=256 \
#     --max_bucket_reso=1024 \
#     --bucket_reso_steps=64 \
#     --save_model_as="safetensors" \
#     --save_precision="fp16" \
#     --save_every_n_epochs=1 \
#     --max_train_epochs=40 \
#     --train_batch_size=12 \
#     --gradient_checkpointing \
#     --network_train_unet_only \
#     --learning_rate=0.00024 \
#     --unet_lr=0.00024 \
#     --text_encoder_lr=0.00001 \
#     --lr_scheduler="constant_with_warmup" \
#     --lr_warmup_steps=100 \
#     --optimizer_type="Lion8bit" \
#     --network_module="lycoris.kohya" \
#     --network_dim=100000 \
#     --network_alpha=100000 \
#     --network_args \
#         conv_dim=100000 \
#         conv_alpha=100000 \
#         algo=lokr \
#         dropout=0 \
#         factor=8 \
#         train_norm=True \
#     --log_with="{LOG_MODULE}" \
#     --logging_dir="{OUTPUT_PATH}/logs" \
#     --caption_extension=".txt" \
#     --shuffle_caption \
#     --keep_tokens=0 \
#     --max_token_length=225 \
#     --seed=1337 \
#     --mixed_precision="fp16" \
#     --xformers \
#     --cache_latents \
#     --cache_latents_to_disk \
#     --persistent_data_loader_workers \
#     --vae_batch_size=4 \
#     --full_fp16


# 使用 lokr 算法训练 SD1.5 多画风(多概念) LoRA, 使用双卡进行训练
# 使用 NovelAI 1 模型进行训练
# 
# 在 SD1.5 中训练 Text Encoder 可以帮助模型更好的区分不同的画风(概念)
# 
# !python -m accelerate.commands.launch \
#     --num_cpu_threads_per_process=1 \
#     --multi_gpu \
#     --num_processes=2 \
#     "{SD_SCRIPTS_PATH}/train_network.py" \
#     --pretrained_model_name_or_path="{SD_MODEL_PATH}/animefull-final-pruned.safetensors" \
#     --vae="{SD_MODEL_PATH}/vae-ft-mse-840000-ema-pruned.safetensors" \
#     --train_data_dir="{INPUT_DATASET_PATH}/sunfish" \
#     --output_name="nai1-sunfish_5" \
#     --output_dir="{OUTPUT_PATH}/nai1-sunfish_5" \
#     --wandb_run_name="Nachoneko" \
#     --log_tracker_name="lora-Nachoneko" \
#     --prior_loss_weight=1 \
#     --resolution="768,768" \
#     --enable_bucket \
#     --min_bucket_reso=256 \
#     --max_bucket_reso=1024 \
#     --bucket_reso_steps=64 \
#     --save_model_as="safetensors" \
#     --save_precision="fp16" \
#     --save_every_n_epochs=1 \
#     --max_train_epochs=40 \
#     --train_batch_size=12 \
#     --gradient_checkpointing \
#     --learning_rate=0.00028 \
#     --unet_lr=0.00028 \
#     --text_encoder_lr=0.000015 \
#     --lr_scheduler="constant_with_warmup" \
#     --lr_warmup_steps=100 \
#     --optimizer_type="Lion8bit" \
#     --network_module="lycoris.kohya" \
#     --network_dim=100000 \
#     --network_alpha=100000 \
#     --network_args \
#         conv_dim=100000 \
#         conv_alpha=100000 \
#         algo=lokr \
#         dropout=0 \
#         factor=8 \
#         train_norm=True \
#     --log_with="{LOG_MODULE}" \
#     --logging_dir="{OUTPUT_PATH}/logs" \
#     --caption_extension=".txt" \
#     --shuffle_caption \
#     --keep_tokens=0 \
#     --max_token_length=225 \
#     --seed=1337 \
#     --mixed_precision="fp16" \
#     --xformers \
#     --cache_latents \
#     --cache_latents_to_disk \
#     --persistent_data_loader_workers \
#     --vae_batch_size=4 \
#     --full_fp16


##########################################################################################
# 下面是 toml 格式的训练命令, 根据上面的训练命令做了格式转换
# 只弄了自己常用的训练参数, 其他的参照下面的例子来改吧
# 
# toml 转换格式如下 (在最前面已经写过一次了, 再写一遍方便对照):
# (1)
# toml 格式:
# pretrained_model_name_or_path = "{SD_MODEL_PATH}/Illustrious-XL-v0.1.safetensors"
# 训练命令格式:
# --pretrained_model_name_or_path="{SD_MODEL_PATH}/Illustrious-XL-v0.1.safetensors"
# 
# (2)
# toml 格式:
# unet_lr = 0.0001
# 训练命令格式:
# --unet_lr=0.0001
# 
# (3)
# toml 格式:
# network_args = [
#     "conv_dim=100000",
#     "conv_alpha=100000",
#     "algo=lokr",
#     "dropout=0",
#     "factor=8",
#     "train_norm=True",
#     "preset=full",
# ]
# 训练命令格式:
# --network_args \
#     conv_dim=100000 \
#     conv_alpha=100000 \
#     algo=lokr \
#     dropout=0 \
#     factor=8 \
#     train_norm=True \
#     preset=full \
# 
# (4)
# toml 格式:
# enable_bucket = true
# 训练命令格式:
# --enable_bucket
# 
# (5)
# toml 格式:
# lowram = false
# 训练命令格式:
# 无对应的训练命令, 也就是不需要填, 因为这个参数的值为 false, 也就是无对应的参数, 如果值为 true, 则对应训练命令中的 --lowram
##########################################################################################



# (自己在用的, toml 格式的版本)
# 使用 lokr 算法训练 XL 画风 LoRA, 使用多卡进行训练
# 该参数也可以用于人物 LoRA 训练
# 
# 在训练多画风 LoRA 或者人物 LoRA 时, 通常会打上触发词
# 当使用了 --network_train_unet_only 后, Text Encoder 虽然不会训练, 但并不影响将触发词训练进 LoRA 模型中
# 并且不训练 Text Encoder 避免 Text Encoder 被炼烂(Text Encoder 比较容易被炼烂)
# 
# 学习率调度器从 cosine_with_restarts 换成 constant_with_warmup, 此时学习率靠优化器(Lion8bit)进行调度
# 拟合速度会更快
# constant_with_warmup 用在大规模的训练上比较好, 但用在小规模训练也有不错的效果
# 如果训练集的图比较少, 重复的图较多, 重复次数较高, 可能容易造成过拟合
# 
# 在 --network_args 设置了 preset, 可以调整训练网络的大小
# 该值默认为 full, 如果使用 attn-mlp 可以得到更小的 LoRA, 但对于难学的概念使用 full 效果会更好
# 
# 可用的预设可阅读文档: https://github.com/KohakuBlueleaf/LyCORIS/blob/main/docs/Preset.md
# 该预设也可以自行编写并指定, 编写例子可查看: https://github.com/KohakuBlueleaf/LyCORIS/blob/main/example_configs/preset_configs/example.toml
# 
# 使用 --optimizer_args 设置 weight_decay 和 betas, 更高的 weight_decay 可以降低拟合程度, 减少过拟合
# 如果拟合程度不够高, 可以提高 --max_train_epochs 的值, 或者适当降低 weight_decay 的值, 可自行测试
# 较小的训练集适合使用较小的值, 如 0.05, 较大的训练集适合用 0.1
# 大概 34 Epoch 会有比较好的效果吧, 不过不好说, 看训练集
# 自己测的时候大概在 26~40 Epoch 之间会出现好结果, 测试了很多炉基本都在这个区间里, 但也不排除意外情况 (训练参数这东西好麻烦啊, 苦い)
# 
# toml_file_path = os.path.join(WORKSPACE, "train_config.toml")
# toml_content = f"""
# pretrained_model_name_or_path = "{SD_MODEL_PATH}/Illustrious-XL-v0.1.safetensors"
# vae = "{SD_MODEL_PATH}/sdxl_fp16_fix_vae.safetensors"
# train_data_dir = "{INPUT_DATASET_PATH}/Nachoneko"
# output_name = "Nachoneko_2"
# output_dir = "{OUTPUT_PATH}/Nachoneko"
# wandb_run_name = "Nachoneko"
# log_tracker_name = "lora-Nachoneko"
# prior_loss_weight = 1
# resolution = "1024,1024"
# enable_bucket = true
# min_bucket_reso = 256
# max_bucket_reso = 4096
# bucket_reso_steps = 64
# save_model_as = "safetensors"
# save_precision = "fp16"
# save_every_n_epochs = 1
# max_train_epochs = 40
# train_batch_size = 6
# gradient_checkpointing = true
# network_train_unet_only = true
# learning_rate = 0.0001
# unet_lr = 0.0001
# text_encoder_lr = 0.00001
# lr_scheduler = "constant_with_warmup"
# lr_warmup_steps = 100
# optimizer_type = "Lion8bit"
# network_module = "lycoris.kohya"
# network_dim = 100000
# network_alpha = 100000
# network_args = [
#     "conv_dim=100000",
#     "conv_alpha=100000",
#     "algo=lokr",
#     "dropout=0",
#     "factor=8",
#     "train_norm=True",
#     "preset=full",
# ]
# optimizer_args = [
#     "weight_decay=0.05",
#     "betas=0.9,0.95",
# ]
# log_with = "{LOG_MODULE}"
# logging_dir = "{OUTPUT_PATH}/logs"
# caption_extension = ".txt"
# shuffle_caption = true
# keep_tokens = 0
# max_token_length = 225
# seed = 1337
# mixed_precision = "fp16"
# xformers = true
# cache_latents = true
# cache_latents_to_disk = true
# persistent_data_loader_workers = true
# vae_batch_size = 4
# full_fp16 = true
# """.strip()
# if not os.path.exists(os.path.dirname(toml_file_path)):
#     os.makedirs(toml_file_path, exist_ok=True)
# with open(toml_file_path, "w", encoding="utf8") as file:
#     file.write(toml_content)
# !python -m accelerate.commands.launch \
#     --num_cpu_threads_per_process=1 \
#     --multi_gpu \
#     --num_processes=2 \
#     "{SD_SCRIPTS_PATH}/sdxl_train_network.py" \
#     --config_file="{toml_file_path}"


# (自己在用的, toml 格式的版本)
# 使用 lokr 算法训练 XL 画风 LoRA, 使用多卡进行训练
# 该参数也可以用于人物 LoRA 训练
# 
# 在训练多画风 LoRA 或者人物 LoRA 时, 通常会打上触发词
# 当使用了 --network_train_unet_only 后, Text Encoder 虽然不会训练, 但并不影响将触发词训练进 LoRA 模型中
# 并且不训练 Text Encoder 避免 Text Encoder 被炼烂(Text Encoder 比较容易被炼烂)
# 
# 学习率调度器从 cosine_with_restarts 换成 constant_with_warmup, 此时学习率靠优化器(Lion8bit)进行调度
# 拟合速度会更快
# constant_with_warmup 用在大规模的训练上比较好, 但用在小规模训练也有不错的效果
# 如果训练集的图比较少, 重复的图较多, 重复次数较高, 可能容易造成过拟合
# 
# 在 --network_args 设置了 preset, 可以调整训练网络的大小
# 该值默认为 full, 如果使用 attn-mlp 可以得到更小的 LoRA, 但对于难学的概念使用 full 效果会更好 (最好还是 full 吧, 其他的预设效果不是很好)
# 
# 可用的预设可阅读文档: https://github.com/KohakuBlueleaf/LyCORIS/blob/main/docs/Preset.md
# 该预设也可以自行编写并指定, 编写例子可查看: https://github.com/KohakuBlueleaf/LyCORIS/blob/main/example_configs/preset_configs/example.toml
# 
# 使用 --optimizer_args 设置 weight_decay 和 betas, 更高的 weight_decay 可以降低拟合程度, 减少过拟合
# 如果拟合程度不够高, 可以提高 --max_train_epochs 的值, 或者适当降低 weight_decay 的值, 可自行测试
# 较小的训练集适合使用较小的值, 如 0.05, 较大的训练集适合用 0.1
# 大概 34 Epoch 会有比较好的效果吧, 不过不好说, 看训练集
# 自己测的时候大概在 26~40 Epoch 之间会出现好结果, 测试了很多炉基本都在这个区间里, 但也不排除意外情况 (训练参数这东西好麻烦啊, 苦い)
# 
# 测试的时候发现 --debiased_estimation_loss 对于训练效果的有些改善
# 这里有个对比: https://licyk.netlify.app/2025/02/10/debiased_estimation_loss_in_stable_diffusion_model_training
# 启用后能提高拟合速度和颜色表现吧, 画风的学习能学得更好
# 但, 肢体崩坏率可能会有点提高, 不过有另一套参数去优化了一下这个问题, 貌似会好一点
# 可能画风会弱化, 所以不是很确定哪个比较好用, 只能自己试了
# debiased estimation loss 有个相关的论文可以看看: https://arxiv.org/abs/2310.08442
# 
# toml_file_path = os.path.join(WORKSPACE, "train_config.toml")
# toml_content = f"""
# pretrained_model_name_or_path = "{SD_MODEL_PATH}/Illustrious-XL-v0.1.safetensors"
# vae = "{SD_MODEL_PATH}/sdxl_fp16_fix_vae.safetensors"
# train_data_dir = "{INPUT_DATASET_PATH}/Nachoneko"
# output_name = "Nachoneko_2"
# output_dir = "{OUTPUT_PATH}/Nachoneko"
# wandb_run_name = "Nachoneko"
# log_tracker_name = "lora-Nachoneko"
# prior_loss_weight = 1
# resolution = "1024,1024"
# enable_bucket = true
# min_bucket_reso = 256
# max_bucket_reso = 4096
# bucket_reso_steps = 64
# save_model_as = "safetensors"
# save_precision = "fp16"
# save_every_n_epochs = 1
# max_train_epochs = 40
# train_batch_size = 6
# gradient_checkpointing = true
# network_train_unet_only = true
# learning_rate = 0.0001
# unet_lr = 0.0001
# text_encoder_lr = 0.00001
# lr_scheduler = "constant_with_warmup"
# lr_warmup_steps = 100
# optimizer_type = "Lion8bit"
# network_module = "lycoris.kohya"
# network_dim = 100000
# network_alpha = 100000
# network_args = [
#     "conv_dim=100000",
#     "conv_alpha=100000",
#     "algo=lokr",
#     "dropout=0",
#     "factor=8",
#     "train_norm=True",
#     "preset=full",
# ]
# optimizer_args = [
#     "weight_decay=0.05",
#     "betas=0.9,0.95",
# ]
# log_with = "{LOG_MODULE}"
# logging_dir = "{OUTPUT_PATH}/logs"
# caption_extension = ".txt"
# shuffle_caption = true
# keep_tokens = 0
# max_token_length = 225
# seed = 1337
# mixed_precision = "fp16"
# xformers = true
# cache_latents = true
# cache_latents_to_disk = true
# persistent_data_loader_workers = true
# debiased_estimation_loss = true
# vae_batch_size = 4
# full_fp16 = true
# """.strip()
# if not os.path.exists(os.path.dirname(toml_file_path)):
#     os.makedirs(toml_file_path, exist_ok=True)
# with open(toml_file_path, "w", encoding="utf8") as file:
#     file.write(toml_content)
# !python -m accelerate.commands.launch \
#     --num_cpu_threads_per_process=1 \
#     --multi_gpu \
#     --num_processes=2 \
#     "{SD_SCRIPTS_PATH}/sdxl_train_network.py" \
#     --config_file="{toml_file_path}"


# (自己在用的, toml 格式的版本)
# 使用 lokr 算法训练 XL 画风 LoRA, 使用多卡进行训练
# 该参数也可以用于人物 LoRA 训练
# 
# 在训练多画风 LoRA 或者人物 LoRA 时, 通常会打上触发词
# 当使用了 --network_train_unet_only 后, Text Encoder 虽然不会训练, 但并不影响将触发词训练进 LoRA 模型中
# 并且不训练 Text Encoder 避免 Text Encoder 被炼烂(Text Encoder 比较容易被炼烂)
# 
# 学习率调度器从 cosine_with_restarts 换成 constant_with_warmup, 此时学习率靠优化器(Lion8bit)进行调度
# 拟合速度会更快
# constant_with_warmup 用在大规模的训练上比较好, 但用在小规模训练也有不错的效果
# 如果训练集的图比较少, 重复的图较多, 重复次数较高, 可能容易造成过拟合
# 
# 在 --network_args 设置了 preset, 可以调整训练网络的大小
# 该值默认为 full, 如果使用 attn-mlp 可以得到更小的 LoRA, 但对于难学的概念使用 full 效果会更好 (最好还是 full 吧, 其他的预设效果不是很好)
# 
# 可用的预设可阅读文档: https://github.com/KohakuBlueleaf/LyCORIS/blob/main/docs/Preset.md
# 该预设也可以自行编写并指定, 编写例子可查看: https://github.com/KohakuBlueleaf/LyCORIS/blob/main/example_configs/preset_configs/example.toml
# 
# 使用 --optimizer_args 设置 weight_decay 和 betas, 更高的 weight_decay 可以降低拟合程度, 减少过拟合
# 如果拟合程度不够高, 可以提高 --max_train_epochs 的值, 或者适当降低 weight_decay 的值, 可自行测试
# 较小的训练集适合使用较小的值, 如 0.05, 较大的训练集适合用 0.1
# 大概 34 Epoch 会有比较好的效果吧, 不过不好说, 看训练集
# 自己测的时候大概在 26~40 Epoch 之间会出现好结果, 测试了很多炉基本都在这个区间里, 但也不排除意外情况 (训练参数这东西好麻烦啊, 苦い)
# 
# 测试的时候发现 --debiased_estimation_loss 对于训练效果的有些改善
# 这里有个对比: https://licyk.netlify.app/2025/02/10/debiased_estimation_loss_in_stable_diffusion_model_training
# 启用后能提高拟合速度和颜色表现吧, 画风的学习能学得更好
# 但, 肢体崩坏率可能会有点提高, 不过有另一套参数去优化了一下这个问题, 貌似会好一点
# 可能画风会弱化, 所以不是很确定哪个比较好用, 只能自己试了
# debiased estimation loss 有个相关的论文可以看看: https://arxiv.org/abs/2310.08442
# 
# 加上 v 预测参数进行训练, 提高模型对暗处和亮处的表现效果, 并且能让模型能够直出纯黑色背景, 画面也更干净
# 相关的论文可以看看: https://arxiv.org/abs/2305.08891
# 
# toml_file_path = os.path.join(WORKSPACE, "train_config.toml")
# toml_content = f"""
# pretrained_model_name_or_path = "{SD_MODEL_PATH}/noobaiXLNAIXL_vPred10Version.safetensors"
# vae = "{SD_MODEL_PATH}/sdxl_fp16_fix_vae.safetensors"
# train_data_dir = "{INPUT_DATASET_PATH}/Nachoneko"
# output_name = "Nachoneko_2"
# output_dir = "{OUTPUT_PATH}/Nachoneko"
# wandb_run_name = "Nachoneko"
# log_tracker_name = "lora-Nachoneko"
# prior_loss_weight = 1
# resolution = "1024,1024"
# enable_bucket = true
# min_bucket_reso = 256
# max_bucket_reso = 4096
# bucket_reso_steps = 64
# save_model_as = "safetensors"
# save_precision = "fp16"
# save_every_n_epochs = 1
# max_train_epochs = 40
# train_batch_size = 6
# gradient_checkpointing = true
# network_train_unet_only = true
# learning_rate = 0.0001
# unet_lr = 0.0001
# text_encoder_lr = 0.00001
# lr_scheduler = "constant_with_warmup"
# lr_warmup_steps = 100
# optimizer_type = "Lion8bit"
# network_module = "lycoris.kohya"
# network_dim = 100000
# network_alpha = 100000
# network_args = [
#     "conv_dim=100000",
#     "conv_alpha=100000",
#     "algo=lokr",
#     "dropout=0",
#     "factor=8",
#     "train_norm=True",
#     "preset=full",
# ]
# optimizer_args = [
#     "weight_decay=0.05",
#     "betas=0.9,0.95",
# ]
# log_with = "{LOG_MODULE}"
# logging_dir = "{OUTPUT_PATH}/logs"
# caption_extension = ".txt"
# shuffle_caption = true
# keep_tokens = 0
# max_token_length = 225
# seed = 1337
# mixed_precision = "fp16"
# xformers = true
# cache_latents = true
# cache_latents_to_disk = true
# persistent_data_loader_workers = true
# debiased_estimation_loss = true
# vae_batch_size = 4
# zero_terminal_snr = true
# v_parameterization = true
# scale_v_pred_loss_like_noise_pred = true
# full_fp16 = true
# """.strip()
# if not os.path.exists(os.path.dirname(toml_file_path)):
#     os.makedirs(toml_file_path, exist_ok=True)
# with open(toml_file_path, "w", encoding="utf8") as file:
#     file.write(toml_content)
# !python -m accelerate.commands.launch \
#     --num_cpu_threads_per_process=1 \
#     --multi_gpu \
#     --num_processes=2 \
#     "{SD_SCRIPTS_PATH}/sdxl_train_network.py" \
#     --config_file="{toml_file_path}"



##########################################################################################
os.chdir(WORKSPACE)
logger.info("离开 sd-scripts 目录")
logger.info("模型训练结束")

## 模型上传
通常不需要修改该单元内容，如果需要修改参数，建议通过上方的参数配置单元进行修改  
5. [← 上一个单元](#模型训练)

In [ ]:
# 模型上传到 HuggingFace / ModelScope, 通常不需要修改, 修改参数建议通过上方的参数配置单元进行修改

# 使用 HuggingFace 上传模型
if USE_HF_TO_SAVE_MODEL:
    logger.info("使用 HuggingFace 保存模型")
    sd_scripts.repo.upload_files_to_repo(
        api_type="huggingface",         # 指定使用 HuggingFace 仓库
        repo_id=HF_REPO_ID,             # HuggingFace 仓库地址
        repo_type=HF_REPO_TYPE,         # HuggingFace 仓库种类
        visibility=HF_REPO_VISIBILITY,  # 当尝试创建仓库时设置仓库的可见性
        upload_path=OUTPUT_PATH,        # 要上传文件的目录
        retry=RETRY,                    # 重试上传的次数
    )

# 使用 ModelScope 上传模型
if USE_MS_TO_SAVE_MODEL:
    logger.info("使用 ModelScope 保存模型")
    sd_scripts.repo.upload_files_to_repo(
        api_type="modelscope",          # 指定使用 ModelScope 仓库
        repo_id=MS_REPO_ID,             # Modelscope 的仓库地址
        repo_type=MS_REPO_TYPE,         # ModelScope 仓库的种类
        visibility=MS_REPO_VISIBILITY,  # 当尝试创建仓库时设置仓库的可见性
        upload_path=OUTPUT_PATH,        # 要上传文件的目录
        retry=RETRY,                    # 重试上传的次数
    )